## US Core Examples Transaction Bundle Maker
### bundle all resources in folder as a transaction using PUT interactions

- python version 3.6+
- get data from folder
- add meta tag "2022-01-US-Core"
- bundle
- update and resolve ids
- validate and save

In [30]:
from json import load, dumps, loads
from IPython import display as D
from requests import get, post, put
from IPython.display import display, Markdown, HTML
from fhirclient.r4models import bundle as B
import fhirclient.r4models.fhirdate as D
import os, uuid
from datetime import datetime

bundle_id = '2022-01-US-Core-upload'
bundle_type = 'transaction'
base_url = 'http://hapi.fhir.org/baseR4'

###  write to file

In [31]:
def write_file(r): # write file
    
    #out_path = '/Users/ehaas/Documents/FHIR/Davinci-DEQM/source/examples/'  # append forward slash
    out_path ='r4'
    print(f'writing to ... {out_path}/Bundle-{bundle_id}.json') 
    with open(f'{out_path}/Bundle-{bundle_id}.json', 'w') as f:
        f.write(r)


### Validate Resource

In [32]:
def validate(resource_type,data): #as json data
 
    #fhir_test_server = 'http://test.fhir.org/r4'
    fhir_test_server = 'http://hapi.fhir.org/baseR4'
    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }

    # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
 
    params = dict(
      # profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient
        )
    
    #   r = requests.post('https://httpbin.org/post', data = {'key':'value'})
    r = post(f'{fhir_test_server}/{resource_type}/$validate', params = params, headers = headers, data = data)
    display(Markdown(f'# {r.status_code} {r.reason}\n\n{r.json()["text"]["div"]}'))
    # view  output
    # return (r.json()["text"]["div"])

### open json file and return as dict

In [33]:
def open_file(f_name): # get files
    with open(f'{in_path}/{f_name}') as f:
        r = f.read()
        return(loads(r))

### Get files in current path

In [34]:
in_path = '/Users/ehaas/.fhir/packages/hl7.fhir.us.core#dev/package/example' #'r4'

files = [x for x in os.listdir(in_path) if x.endswith(".json") and x not in [".index.json"]]
for i,f in enumerate(sorted(files)):
    print(f"{i+1}) {f}")


1) AllergyIntolerance-example.json
2) Bundle-66c8856b-ba11-4876-8aa8-467aad8c11a2.json
3) Bundle-c887e62f-6166-419f-8268-b5ecd6c7b901.json
4) Bundle-uscore-mo3.json
5) CarePlan-colonoscopy.json
6) CareTeam-example.json
7) Condition-condition-SDOH-example.json
8) Condition-example.json
9) Condition-hc1.json
10) Device-udi-1.json
11) Device-udi-2.json
12) Device-udi-3.json
13) DiagnosticReport-bone-density-report.json
14) DiagnosticReport-cardiology-report.json
15) DiagnosticReport-cbc.json
16) DiagnosticReport-chest-xray-report.json
17) DiagnosticReport-metabolic-panel.json
18) DiagnosticReport-urinalysis.json
19) DocumentReference-episode-summary.json
20) Encounter-1036.json
21) Encounter-delivery.json
22) Encounter-example-1.json
23) Goal-goal-1.json
24) Goal-goal-sdoh-2.json
25) Immunization-imm-1.json
26) Location-hl7east.json
27) Location-hospital.json
28) Media-ekg-strip.json
29) Medication-uscore-med1.json
30) Medication-uscore-med2.json
31) MedicationRequest-self-tylenol.json
32

### create Bundle 'b'  change the id for unique Bundles!!!

In [35]:
b = B.Bundle()
b.id =  bundle_id
b.type = bundle_type
b.timestamp = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
b.as_json()

{'id': '2022-01-US-Core-upload',
 'timestamp': '2022-01-03T20:45:39.209055Z',
 'type': 'transaction',
 'resourceType': 'Bundle'}

### Add resources to bundle

 - add tag to each resource first
 - remove text elements

In [36]:
def get_recursively(search_dict, field):
    """
    Takes a dict with nested lists and dicts,
    and searches all dicts for a key of the field
    provided.
    """
    fields_found = []

    for key, value in search_dict.items():

        if key == field:
            fields_found.append(value)

        elif isinstance(value, dict):
            results = get_recursively(value, field)
            for result in results:
                fields_found.append(result)

        elif isinstance(value, list):
            #print(f'{value} is list')
            for item in value:
                if isinstance(item, dict):
                    more_results = get_recursively(item, field)
                    for another_result in more_results:
                        fields_found.append(another_result)

    return fields_found

In [37]:
b.entry = []
type_list = []
ref_map = {}

for i in files:
    r = open_file(i)
    if r["resourceType"] in ["Bundle"]: # no bundles for now!
        continue
    
    #update ids to new unique ids for upload
    type_list.append(r["resourceType"])
    old_ref = f'{r["resourceType"]}/{r["id"]}'
    new_id = f'202201uscore-{r["resourceType"].lower()}-example-{type_list.count(r["resourceType"])}'
    new_ref = f'{r["resourceType"]}/{new_id}'
    ref_map[old_ref] = new_ref

    r.pop('text', None) # remove text elements from Bundle
    r['meta']['tag']= [{"code":"202201uscore",}] # add meta tag
    r["id"] = new_id
    
    e = B.BundleEntry(dict
        (
        resource = r,
        fullUrl = f'{base_url}/{new_ref}'
        )
      )
    if bundle_type in ['transaction', 'batch']:
        e.request = B.BundleEntryRequest(dict
                    (
                    method = 'PUT',
                    url = new_ref,
                    )
                    )

    b.entry.append(e)  


### Resort bundle for transaction
 - resort for transaction to make sure all the referred to resources are before the referring ones
 - rename ids for PUT to server


In [38]:

transaction_order = dict(
Patient = 0,
Organization = 1,
Practitioner = 2,
Location = 4,
PractitionerRole = 5,
Encounter = 6,
Observation = 7,
Media = 8,
Medication = 9,
RelatedPerson = 10,
)

b.entry.sort(key=lambda x: transaction_order.get(x.resource.resource_type,11))
# for i,r in enumerate(b.entry):
#     print(f'{i+1})',r.resource.resource_type)

# change all the references to the new ids
b_json = dumps(b.as_json(),indent=3,)
for old_ref, new_ref in ref_map.items():
     b_json = b_json.replace(old_ref,new_ref)  
# display
print(f'{b_json[0:1000]}\n...\n{b_json[-1000:]}')

{
   "id": "2022-01-US-Core-upload",
   "entry": [
      {
         "fullUrl": "http://hapi.fhir.org/baseR4/Patient/202201uscore-patient-example-1",
         "request": {
            "method": "PUT",
            "url": "Patient/202201uscore-patient-example-1"
         },
         "resource": {
            "id": "202201uscore-patient-example-1",
            "meta": {
               "extension": [
                  {
                     "url": "http://hl7.org/fhir/StructureDefinition/instance-name",
                     "valueString": "Patient Example"
                  },
                  {
                     "url": "http://hl7.org/fhir/StructureDefinition/instance-description",
                     "valueMarkdown": "This is a patient example for the *US Core Patient Profile*."
                  }
               ],
               "profile": [
                  "http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient"
               ],
               "tag": [
                

### Validate as JSON String

In [39]:
data = b_json
resource_type = b.resource_type

validate(resource_type,data)

# 412 Precondition Failed

<div xmlns="http://www.w3.org/1999/xhtml"><h1>Operation Outcome</h1><table border="0"><tr><td style="font-weight: bold;">WARNING</td><td>[Bundle.entry[0].resource.ofType(Patient), Line 12, Col 22]</td><td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[0].resource.ofType(Patient).meta.tag[0], Line 27, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[0].resource.ofType(Patient).extension[2].value.ofType(code), Line 118, Col 17]</td>
				<td><pre>The value provided ('F') is not in the value set http://hl7.org/fhir/us/core/ValueSet/birthsex (http://hl7.org/fhir/us/core/ValueSet/birthsex), and a code is required from this value set) (error message = Unable to validate code null#F - Unknown code null#F. Code validation occurred using a ValueSet expansion that was pre-calculated at 2021-10-19T16:53:12.554+00:00 (76 days ago))</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[0].resource.ofType(Patient).extension[3], Line 119, Col 17]</td>
				<td><pre>Unknown extension http://hl7.org/fhir/us/core/StructureDefinition/us-core-genderIdentity</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[0].resource.ofType(Patient).extension[3], Line 119, Col 17]</td>
				<td><pre>This element does not match any known slice defined in the profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[1].resource.ofType(Patient), Line 230, Col 22]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[1].resource.ofType(Patient).meta.tag[0], Line 245, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[1].resource.ofType(Patient).extension[2].value.ofType(code), Line 288, Col 17]</td>
				<td><pre>The value provided ('M') is not in the value set http://hl7.org/fhir/us/core/ValueSet/birthsex (http://hl7.org/fhir/us/core/ValueSet/birthsex), and a code is required from this value set) (error message = Unable to validate code null#M - Unknown code null#M. Code validation occurred using a ValueSet expansion that was pre-calculated at 2021-10-19T16:53:12.554+00:00 (76 days ago))</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[2].resource.ofType(Patient), Line 347, Col 22]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[2].resource.ofType(Patient).meta.tag[0], Line 362, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[2].resource.ofType(Patient).extension[2].value.ofType(code), Line 405, Col 17]</td>
				<td><pre>The value provided ('M') is not in the value set http://hl7.org/fhir/us/core/ValueSet/birthsex (http://hl7.org/fhir/us/core/ValueSet/birthsex), and a code is required from this value set) (error message = Unable to validate code null#M - Unknown code null#M. Code validation occurred using a ValueSet expansion that was pre-calculated at 2021-10-19T16:53:12.554+00:00 (76 days ago))</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[3].resource.ofType(Organization), Line 464, Col 22]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[3].resource.ofType(Organization).meta.tag[0], Line 479, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[4].resource.ofType(Organization), Line 539, Col 22]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[4].resource.ofType(Organization).meta.tag[0], Line 554, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[4].resource.ofType(Organization).contained[0].ofType(Endpoint), Line 560, Col 17]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[4].resource.ofType(Organization).identifier[1], Line 610, Col 17]</td>
				<td><pre>This element does not match any known slice defined in the profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[5].resource.ofType(Organization), Line 639, Col 22]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[5].resource.ofType(Organization).meta.tag[0], Line 654, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[5].resource.ofType(Organization).identifier[1], Line 676, Col 17]</td>
				<td><pre>This element does not match any known slice defined in the profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[6].resource.ofType(Practitioner), Line 714, Col 22]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[6].resource.ofType(Practitioner).meta.tag[0], Line 729, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[6].resource.ofType(Practitioner).identifier[1], Line 750, Col 17]</td>
				<td><pre>This element does not match any known slice defined in the profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[7].resource.ofType(Practitioner), Line 777, Col 22]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[7].resource.ofType(Practitioner).meta.tag[0], Line 792, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[7].resource.ofType(Practitioner).identifier[1], Line 813, Col 17]</td>
				<td><pre>This element does not match any known slice defined in the profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[8].resource.ofType(Location), Line 882, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[8].resource.ofType(Location).meta.tag[0], Line 868, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[9].resource.ofType(Location), Line 945, Col 26]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[9].resource.ofType(Location).meta.tag[0], Line 920, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[10].resource.ofType(Encounter), Line 1001, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[10].resource.ofType(Encounter).meta.tag[0], Line 982, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[10].resource.ofType(Encounter).type[0].coding[0], Line 1008, Col 23]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#177184002 - Code can not be found in CodeSystem for 'http://snomed.info/sct#177184002'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[10].resource.ofType(Encounter).subject, Line 1002, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[10].resource.ofType(Encounter).type[0], Line 1006, Col 17]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-encounter-type (http://hl7.org/fhir/us/core/ValueSet/us-core-encounter-type), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://snomed.info/sct#177184002)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[10].resource.ofType(Encounter).subject, Line 1002, Col 41]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[10].resource.ofType(Encounter).subject, Line 1002, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[11].resource.ofType(Encounter), Line 1073, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[11].resource.ofType(Encounter).meta.tag[0], Line 1043, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[11].resource.ofType(Encounter).type[0].coding[0], Line 1079, Col 23]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#261665006 - Code can not be found in CodeSystem for 'http://snomed.info/sct#261665006'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[11].resource.ofType(Encounter).subject, Line 1075, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[11].resource.ofType(Encounter).type[0], Line 1077, Col 17]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-encounter-type (http://hl7.org/fhir/us/core/ValueSet/us-core-encounter-type), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://snomed.info/sct#261665006)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[11].resource.ofType(Encounter).subject, Line 1075, Col 14]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[11].resource.ofType(Encounter).subject, Line 1075, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[12].resource.ofType(Encounter), Line 1130, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[12].resource.ofType(Encounter).meta.tag[0], Line 1115, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[12].resource.ofType(Encounter).subject, Line 1132, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[12].resource.ofType(Encounter).type[0], Line 1134, Col 17]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-encounter-type (http://hl7.org/fhir/us/core/ValueSet/us-core-encounter-type), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://www.ama-assn.org/go/cpt#99201)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[12].resource.ofType(Encounter).subject, Line 1132, Col 14]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[12].resource.ofType(Encounter).subject, Line 1132, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[13].resource.ofType(Observation), Line 1206, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[13].resource.ofType(Observation).meta.tag[0], Line 1170, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[13].resource.ofType(Observation).code.coding[0], Line 1189, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#9279-1 - Code can not be found in CodeSystem for 'http://loinc.org#9279-1'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[13].resource.ofType(Observation).subject, Line 1203, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[13].resource.ofType(Observation).value.ofType(Quantity), Line 1207, Col 31]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#/min - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#/min'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[13].resource.ofType(Observation), Line 1206, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[13].resource.ofType(Observation).code, Line 1188, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#9279-1)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[13].resource.ofType(Observation).subject, Line 1203, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[13].resource.ofType(Observation).subject, Line 1203, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[14].resource.ofType(Observation), Line 1278, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[14].resource.ofType(Observation).meta.tag[0], Line 1239, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[14].resource.ofType(Observation).code.coding[0], Line 1259, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#5770-3 - Code can not be found in CodeSystem for 'http://loinc.org#5770-3'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[14].resource.ofType(Observation).subject, Line 1275, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[14].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 1280, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#260385009 - Code can not be found in CodeSystem for 'http://snomed.info/sct#260385009'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[14].resource.ofType(Observation), Line 1278, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[14].resource.ofType(Observation).code, Line 1258, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#5770-3)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[14].resource.ofType(Observation).subject, Line 1275, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[14].resource.ofType(Observation).subject, Line 1275, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[15].resource.ofType(Observation), Line 1354, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[15].resource.ofType(Observation).meta.tag[0], Line 1315, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[15].resource.ofType(Observation).code.coding[0], Line 1335, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#5799-2 - Code can not be found in CodeSystem for 'http://loinc.org#5799-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[15].resource.ofType(Observation).subject, Line 1351, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[15].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 1356, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#260348001 - Code can not be found in CodeSystem for 'http://snomed.info/sct#260348001'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[15].resource.ofType(Observation), Line 1354, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[15].resource.ofType(Observation).code, Line 1334, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#5799-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[15].resource.ofType(Observation).subject, Line 1351, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[15].resource.ofType(Observation).subject, Line 1351, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[16].resource.ofType(Observation), Line 1451, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[16].resource.ofType(Observation).meta.tag[0], Line 1391, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[16].resource.ofType(Observation).code.coding[0], Line 1411, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#770-8 - Code can not be found in CodeSystem for 'http://loinc.org#770-8'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[16].resource.ofType(Observation).subject, Line 1448, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[16].resource.ofType(Observation).referenceRange[0].low, Line 1429, Col 34]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#% - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#%'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[16].resource.ofType(Observation).referenceRange[0].high, Line 1423, Col 34]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#% - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#%'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[16].resource.ofType(Observation), Line 1451, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[16].resource.ofType(Observation).code, Line 1410, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#770-8)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[16].resource.ofType(Observation).subject, Line 1448, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[16].resource.ofType(Observation).subject, Line 1448, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[17].resource.ofType(Observation), Line 1522, Col 28]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[17].resource.ofType(Observation).meta.tag[0], Line 1483, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[17].resource.ofType(Observation).code.coding[0], Line 1503, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#5787-7 - Code can not be found in CodeSystem for 'http://loinc.org#5787-7'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[17].resource.ofType(Observation).subject, Line 1519, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[17].resource.ofType(Observation).value.ofType(Range).low, Line 1530, Col 36]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#/[HPF] - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#/[HPF]'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[17].resource.ofType(Observation).value.ofType(Range).high, Line 1524, Col 36]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#/[HPF] - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#/[HPF]'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[17].resource.ofType(Observation), Line 1522, Col 28]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[17].resource.ofType(Observation).code, Line 1502, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#5787-7)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[17].resource.ofType(Observation).subject, Line 1519, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[17].resource.ofType(Observation).subject, Line 1519, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[18].resource.ofType(Observation), Line 1594, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[18].resource.ofType(Observation).meta.tag[0], Line 1562, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[18].resource.ofType(Observation).code.coding[0], Line 1581, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#44967-8 - Code can not be found in CodeSystem for 'http://loinc.org#44967-8'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[18].resource.ofType(Observation).subject, Line 1595, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[18].resource.ofType(Observation), Line 1594, Col 25]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[18].resource.ofType(Observation).meta.profile[0], Line 1594, Col 25]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[19].resource.ofType(Observation), Line 1695, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[19].resource.ofType(Observation).meta.tag[0], Line 1625, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[19].resource.ofType(Observation).code.coding[0], Line 1644, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#85354-9 - Code can not be found in CodeSystem for 'http://loinc.org#85354-9'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[19].resource.ofType(Observation).subject, Line 1696, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[19].resource.ofType(Observation).component[0].code.coding[0], Line 1656, Col 26]</td>
				<td><pre>Unable to validate code http://loinc.org#8480-6 - Code can not be found in CodeSystem for 'http://loinc.org#8480-6'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[19].resource.ofType(Observation).component[0].value.ofType(Quantity), Line 1665, Col 39]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mm[Hg] - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mm[Hg]'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[19].resource.ofType(Observation).component[1].code.coding[0], Line 1674, Col 26]</td>
				<td><pre>Unable to validate code http://loinc.org#8462-4 - Code can not be found in CodeSystem for 'http://loinc.org#8462-4'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[19].resource.ofType(Observation).component[1].value.ofType(Quantity), Line 1683, Col 39]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mm[Hg] - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mm[Hg]'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[19].resource.ofType(Observation), Line 1695, Col 25]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[19].resource.ofType(Observation).code, Line 1643, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#85354-9)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[19].resource.ofType(Observation).subject, Line 1696, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[19].resource.ofType(Observation).subject, Line 1696, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[19].resource.ofType(Observation).component[0].code, Line 1655, Col 33]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#8480-6)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[19].resource.ofType(Observation).component[1].code, Line 1673, Col 33]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#8462-4)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[20].resource.ofType(Observation), Line 1765, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[20].resource.ofType(Observation).meta.tag[0], Line 1726, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[20].resource.ofType(Observation).code.coding[0], Line 1746, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#13945-1 - Code can not be found in CodeSystem for 'http://loinc.org#13945-1'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[20].resource.ofType(Observation).subject, Line 1762, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[20].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 1767, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#84638005 - Code can not be found in CodeSystem for 'http://snomed.info/sct#84638005'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[20].resource.ofType(Observation), Line 1765, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[20].resource.ofType(Observation).code, Line 1745, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#13945-1)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[20].resource.ofType(Observation).subject, Line 1762, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[20].resource.ofType(Observation).subject, Line 1762, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[21].resource.ofType(Observation), Line 1841, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[21].resource.ofType(Observation).meta.tag[0], Line 1802, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[21].resource.ofType(Observation).code.coding[0], Line 1822, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#25428-4 - Code can not be found in CodeSystem for 'http://loinc.org#25428-4'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[21].resource.ofType(Observation).subject, Line 1838, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[21].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 1843, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#260385009 - Code can not be found in CodeSystem for 'http://snomed.info/sct#260385009'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[21].resource.ofType(Observation), Line 1841, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[21].resource.ofType(Observation).code, Line 1821, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#25428-4)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[21].resource.ofType(Observation).subject, Line 1838, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[21].resource.ofType(Observation).subject, Line 1838, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[22].resource.ofType(Observation), Line 1913, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[22].resource.ofType(Observation).meta.tag[0], Line 1877, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[22].resource.ofType(Observation).code.coding[0], Line 1896, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#44974-4 - Code can not be found in CodeSystem for 'http://loinc.org#44974-4'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[22].resource.ofType(Observation).subject, Line 1910, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[22].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 1915, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#LA11841-6 - Code can not be found in CodeSystem for 'http://loinc.org#LA11841-6'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[22].resource.ofType(Observation), Line 1913, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[22].resource.ofType(Observation).meta.profile[0], Line 1913, Col 38]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[23].resource.ofType(Observation), Line 1985, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[23].resource.ofType(Observation).meta.tag[0], Line 1949, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[23].resource.ofType(Observation).code.coding[0], Line 1968, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#44973-6 - Code can not be found in CodeSystem for 'http://loinc.org#44973-6'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[23].resource.ofType(Observation).subject, Line 1982, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[23].resource.ofType(Observation).value.ofType(Quantity), Line 1986, Col 29]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#ms - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#ms'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[23].resource.ofType(Observation), Line 1985, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[23].resource.ofType(Observation).meta.profile[0], Line 1985, Col 31]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[24].resource.ofType(Observation), Line 2057, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[24].resource.ofType(Observation).meta.tag[0], Line 2018, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[24].resource.ofType(Observation).code.coding[0], Line 2038, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#5778-6 - Code can not be found in CodeSystem for 'http://loinc.org#5778-6'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[24].resource.ofType(Observation).subject, Line 2054, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[24].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 2059, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#371244009 - Code can not be found in CodeSystem for 'http://snomed.info/sct#371244009'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[24].resource.ofType(Observation), Line 2057, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[24].resource.ofType(Observation).code, Line 2037, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#5778-6)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[24].resource.ofType(Observation).subject, Line 2054, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[24].resource.ofType(Observation).subject, Line 2054, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[25].resource.ofType(Observation), Line 2129, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[25].resource.ofType(Observation).meta.tag[0], Line 2093, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[25].resource.ofType(Observation).code.coding[0], Line 2112, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#8310-5 - Code can not be found in CodeSystem for 'http://loinc.org#8310-5'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[25].resource.ofType(Observation).subject, Line 2126, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[25].resource.ofType(Observation).value.ofType(Quantity), Line 2130, Col 30]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#Cel - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#Cel'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[25].resource.ofType(Observation), Line 2129, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[25].resource.ofType(Observation).code, Line 2111, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#8310-5)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[25].resource.ofType(Observation).subject, Line 2126, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[25].resource.ofType(Observation).subject, Line 2126, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[25].resource.ofType(Observation).value.ofType(Quantity).code, Line 2130, Col 30]</td>
				<td><pre>The value provided ('Cel') is not in the value set http://hl7.org/fhir/ValueSet/ucum-bodytemp|4.0.1 (http://hl7.org/fhir/ValueSet/ucum-bodytemp), and a code is required from this value set) (error message = Unknown code 'Cel' for in-memory expansion of ValueSet 'http://hl7.org/fhir/ValueSet/ucum-bodytemp')</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[26].resource.ofType(Observation), Line 2259, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[26].resource.ofType(Observation).meta.tag[0], Line 2161, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[26].resource.ofType(Observation).code.coding[0], Line 2180, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#2708-6 - Code can not be found in CodeSystem for 'http://loinc.org#2708-6'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[26].resource.ofType(Observation).code.coding[1], Line 2185, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#59408-5 - Code can not be found in CodeSystem for 'http://loinc.org#59408-5'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[26].resource.ofType(Observation).subject, Line 2258, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[26].resource.ofType(Observation).value.ofType(Quantity), Line 2260, Col 28]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#% - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#%'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[26].resource.ofType(Observation).referenceRange[0].low, Line 2248, Col 34]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#% - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#%'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[26].resource.ofType(Observation).referenceRange[0].high, Line 2242, Col 34]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#% - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#%'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[26].resource.ofType(Observation).component[0].code.coding[0], Line 2201, Col 26]</td>
				<td><pre>Unable to validate code http://loinc.org#3151-8 - Code can not be found in CodeSystem for 'http://loinc.org#3151-8'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[26].resource.ofType(Observation).component[0].value.ofType(Quantity), Line 2210, Col 38]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#L/min - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#L/min'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[26].resource.ofType(Observation), Line 2259, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[26].resource.ofType(Observation).code, Line 2179, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#2708-6, http://loinc.org#59408-5, urn:iso:std:iso:11073:10101#150456)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[26].resource.ofType(Observation).code.coding[2], Line 2190, Col 20]</td>
				<td><pre>This element does not match any known slice defined in the profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-pulse-oximetry</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[26].resource.ofType(Observation).subject, Line 2258, Col 14]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[26].resource.ofType(Observation).subject, Line 2258, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[26].resource.ofType(Observation).component[0].code, Line 2200, Col 33]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#3151-8)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[27].resource.ofType(Observation), Line 2322, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[27].resource.ofType(Observation).meta.tag[0], Line 2291, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[27].resource.ofType(Observation).code.coding[0], Line 2309, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#76690-7 - Code can not be found in CodeSystem for 'http://loinc.org#76690-7'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[27].resource.ofType(Observation).subject, Line 2321, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[27].resource.ofType(Observation), Line 2322, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[27].resource.ofType(Observation).meta.profile[0], Line 2322, Col 38]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-sexual-orientation' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[28].resource.ofType(Observation), Line 2419, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[28].resource.ofType(Observation).meta.tag[0], Line 2359, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[28].resource.ofType(Observation).code.coding[0], Line 2379, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#2823-3 - Code can not be found in CodeSystem for 'http://loinc.org#2823-3'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[28].resource.ofType(Observation).subject, Line 2416, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[28].resource.ofType(Observation).referenceRange[0].low, Line 2397, Col 39]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mmol/L - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mmol/L'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[28].resource.ofType(Observation).referenceRange[0].high, Line 2391, Col 39]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mmol/L - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mmol/L'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[28].resource.ofType(Observation), Line 2419, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[28].resource.ofType(Observation).code, Line 2378, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#2823-3)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[28].resource.ofType(Observation).subject, Line 2416, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[28].resource.ofType(Observation).subject, Line 2416, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[29].resource.ofType(Observation), Line 2483, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[29].resource.ofType(Observation).meta.tag[0], Line 2450, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[29].resource.ofType(Observation).code.coding[0], Line 2469, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#72166-2 - Code can not be found in CodeSystem for 'http://loinc.org#72166-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[29].resource.ofType(Observation).subject, Line 2480, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[29].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 2485, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#428041000124106 - Code can not be found in CodeSystem for 'http://snomed.info/sct#428041000124106'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[29].resource.ofType(Observation), Line 2483, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[29].resource.ofType(Observation).code, Line 2468, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-smoking-status-observation-codes (http://hl7.org/fhir/us/core/ValueSet/us-core-smoking-status-observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#72166-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[29].resource.ofType(Observation).subject, Line 2480, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[29].resource.ofType(Observation).subject, Line 2480, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[29].resource.ofType(Observation).value.ofType(CodeableConcept), Line 2484, Col 27]</td>
				<td><pre>ValueSet http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113883.11.20.9.38 not found by validator</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[30].resource.ofType(Observation), Line 2579, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[30].resource.ofType(Observation).meta.tag[0], Line 2519, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[30].resource.ofType(Observation).code.coding[0], Line 2539, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#2951-2 - Code can not be found in CodeSystem for 'http://loinc.org#2951-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[30].resource.ofType(Observation).subject, Line 2576, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[30].resource.ofType(Observation).referenceRange[0].low, Line 2557, Col 39]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mmol/L - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mmol/L'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[30].resource.ofType(Observation).referenceRange[0].high, Line 2551, Col 39]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mmol/L - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mmol/L'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[30].resource.ofType(Observation), Line 2579, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[30].resource.ofType(Observation).code, Line 2538, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#2951-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[30].resource.ofType(Observation).subject, Line 2576, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[30].resource.ofType(Observation).subject, Line 2576, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[31].resource.ofType(Observation), Line 2671, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[31].resource.ofType(Observation).meta.tag[0], Line 2611, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[31].resource.ofType(Observation).code.coding[0], Line 2631, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#2160-0 - Code can not be found in CodeSystem for 'http://loinc.org#2160-0'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[31].resource.ofType(Observation).subject, Line 2668, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[31].resource.ofType(Observation).referenceRange[0].low, Line 2649, Col 38]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mg/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mg/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[31].resource.ofType(Observation).referenceRange[0].high, Line 2643, Col 38]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mg/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mg/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[31].resource.ofType(Observation), Line 2671, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[31].resource.ofType(Observation).code, Line 2630, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#2160-0)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[31].resource.ofType(Observation).subject, Line 2668, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[31].resource.ofType(Observation).subject, Line 2668, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[32].resource.ofType(Observation), Line 2743, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[32].resource.ofType(Observation).meta.tag[0], Line 2702, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[32].resource.ofType(Observation).code.coding[0], Line 2721, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#2708-6 - Code can not be found in CodeSystem for 'http://loinc.org#2708-6'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[32].resource.ofType(Observation).code.coding[1], Line 2726, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#59408-5 - Code can not be found in CodeSystem for 'http://loinc.org#59408-5'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[32].resource.ofType(Observation).subject, Line 2740, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[32].resource.ofType(Observation).value.ofType(Quantity), Line 2744, Col 28]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#% - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#%'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[32].resource.ofType(Observation), Line 2743, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[32].resource.ofType(Observation).code, Line 2720, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#2708-6, http://loinc.org#59408-5)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[32].resource.ofType(Observation).subject, Line 2740, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[32].resource.ofType(Observation).subject, Line 2740, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[33].resource.ofType(Observation), Line 2816, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[33].resource.ofType(Observation).meta.tag[0], Line 2775, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[33].resource.ofType(Observation).code.coding[0], Line 2794, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#77606-2 - Code can not be found in CodeSystem for 'http://loinc.org#77606-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[33].resource.ofType(Observation).subject, Line 2813, Col 44]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-2 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[33].resource.ofType(Observation).value.ofType(Quantity), Line 2817, Col 28]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#% - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#%'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[33].resource.ofType(Observation), Line 2816, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[33].resource.ofType(Observation).code, Line 2793, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#77606-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[33].resource.ofType(Observation).subject, Line 2813, Col 44]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-2 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[33].resource.ofType(Observation).subject, Line 2813, Col 44]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-2 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[34].resource.ofType(Observation), Line 2884, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[34].resource.ofType(Observation).meta.tag[0], Line 2848, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[34].resource.ofType(Observation).code.coding[0], Line 2867, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#69428-1 - Code can not be found in CodeSystem for 'http://loinc.org#69428-1'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[34].resource.ofType(Observation).subject, Line 2881, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[34].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 2886, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#LA9634-2 - Code can not be found in CodeSystem for 'http://loinc.org#LA9634-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[34].resource.ofType(Observation), Line 2884, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[34].resource.ofType(Observation).meta.profile[0], Line 2884, Col 38]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[35].resource.ofType(Observation), Line 2956, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[35].resource.ofType(Observation).meta.tag[0], Line 2920, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[35].resource.ofType(Observation).code.coding[0], Line 2939, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#46278-8 - Code can not be found in CodeSystem for 'http://loinc.org#46278-8'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[35].resource.ofType(Observation).subject, Line 2953, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[35].resource.ofType(Observation).value.ofType(Quantity), Line 2957, Col 32]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#g/cm2 - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#g/cm2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[35].resource.ofType(Observation), Line 2956, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[35].resource.ofType(Observation).meta.profile[0], Line 2956, Col 31]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-imaging' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[36].resource.ofType(Observation), Line 3037, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[36].resource.ofType(Observation).meta.tag[0], Line 2988, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[36].resource.ofType(Observation).category[0], Line 2994, Col 17]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-category (http://hl7.org/fhir/ValueSet/observation-category), and a coding is recommended to come from this value set) (codes = http://loinc.org#LG41762-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[36].resource.ofType(Observation).category[0].coding[0], Line 2996, Col 23]</td>
				<td><pre>Unable to validate code http://loinc.org#LG41762-2 - Code can not be found in CodeSystem for 'http://loinc.org#LG41762-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[36].resource.ofType(Observation).code.coding[0], Line 3015, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#88122-7 - Code can not be found in CodeSystem for 'http://loinc.org#88122-7'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[36].resource.ofType(Observation).subject, Line 3036, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[36].resource.ofType(Observation).performer[0], Line 3029, Col 17]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[36].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 3039, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#LA28397-0 - Code can not be found in CodeSystem for 'http://loinc.org#LA28397-0'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[36].resource.ofType(Observation).meta.profile[0], Line 3037, Col 38]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-screening-response' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[37].resource.ofType(Observation), Line 3160, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[37].resource.ofType(Observation).meta.tag[0], Line 3072, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[37].resource.ofType(Observation).category[0], Line 3078, Col 17]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-category (http://hl7.org/fhir/ValueSet/observation-category), and a coding is recommended to come from this value set) (codes = http://loinc.org#LG41762-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[37].resource.ofType(Observation).category[0].coding[0], Line 3080, Col 23]</td>
				<td><pre>Unable to validate code http://loinc.org#LG41762-2 - Code can not be found in CodeSystem for 'http://loinc.org#LG41762-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[37].resource.ofType(Observation).code.coding[0], Line 3099, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#93031-3 - Code can not be found in CodeSystem for 'http://loinc.org#93031-3'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[37].resource.ofType(Observation).subject, Line 3162, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[37].resource.ofType(Observation).performer[0], Line 3155, Col 17]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[37].resource.ofType(Observation).component[0].code.coding[0], Line 3110, Col 26]</td>
				<td><pre>Unable to validate code http://loinc.org#93031-3 - Code can not be found in CodeSystem for 'http://loinc.org#93031-3'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[37].resource.ofType(Observation).component[0].value.ofType(CodeableConcept).coding[0], Line 3119, Col 26]</td>
				<td><pre>Unable to validate code http://loinc.org#LA30126-9 - Code can not be found in CodeSystem for 'http://loinc.org#LA30126-9'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[37].resource.ofType(Observation).component[1].code.coding[0], Line 3130, Col 26]</td>
				<td><pre>Unable to validate code http://loinc.org#93031-3 - Code can not be found in CodeSystem for 'http://loinc.org#93031-3'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[37].resource.ofType(Observation).component[1].value.ofType(CodeableConcept).coding[0], Line 3139, Col 26]</td>
				<td><pre>Unable to validate code http://loinc.org#LA30127-7 - Code can not be found in CodeSystem for 'http://loinc.org#LA30127-7'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[37].resource.ofType(Observation).meta.profile[0], Line 3160, Col 25]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-screening-response' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[38].resource.ofType(Observation), Line 3227, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[38].resource.ofType(Observation).meta.tag[0], Line 3189, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[38].resource.ofType(Observation).code.coding[0], Line 3208, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#8289-1 - Code can not be found in CodeSystem for 'http://loinc.org#8289-1'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[38].resource.ofType(Observation).subject, Line 3224, Col 44]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-2 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[38].resource.ofType(Observation).value.ofType(Quantity), Line 3228, Col 28]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#% - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#%'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[38].resource.ofType(Observation), Line 3227, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[38].resource.ofType(Observation).code, Line 3207, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#8289-1)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[38].resource.ofType(Observation).subject, Line 3224, Col 44]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-2 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[38].resource.ofType(Observation).subject, Line 3224, Col 44]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-2 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[39].resource.ofType(Observation), Line 3299, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[39].resource.ofType(Observation).meta.tag[0], Line 3260, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[39].resource.ofType(Observation).code.coding[0], Line 3280, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#5794-3 - Code can not be found in CodeSystem for 'http://loinc.org#5794-3'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[39].resource.ofType(Observation).subject, Line 3296, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[39].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 3301, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#260405006 - Code can not be found in CodeSystem for 'http://snomed.info/sct#260405006'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[39].resource.ofType(Observation), Line 3299, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[39].resource.ofType(Observation).code, Line 3279, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#5794-3)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[39].resource.ofType(Observation).subject, Line 3296, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[39].resource.ofType(Observation).subject, Line 3296, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[40].resource.ofType(Observation), Line 3376, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[40].resource.ofType(Observation).meta.tag[0], Line 3335, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[40].resource.ofType(Observation).code.coding[0], Line 3354, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#8306-3 - Code can not be found in CodeSystem for 'http://loinc.org#8306-3'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[40].resource.ofType(Observation).code.coding[1], Line 3359, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#8302-2 - Code can not be found in CodeSystem for 'http://loinc.org#8302-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[40].resource.ofType(Observation).subject, Line 3373, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[40].resource.ofType(Observation).value.ofType(Quantity), Line 3377, Col 29]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#cm - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#cm'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[40].resource.ofType(Observation), Line 3376, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[40].resource.ofType(Observation).code, Line 3353, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#8306-3, http://loinc.org#8302-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[40].resource.ofType(Observation).subject, Line 3373, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[40].resource.ofType(Observation).subject, Line 3373, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[40].resource.ofType(Observation).value.ofType(Quantity).code, Line 3377, Col 29]</td>
				<td><pre>The value provided ('cm') is not in the value set http://hl7.org/fhir/ValueSet/ucum-bodylength|4.0.1 (http://hl7.org/fhir/ValueSet/ucum-bodylength), and a code is required from this value set) (error message = Unknown code 'cm' for in-memory expansion of ValueSet 'http://hl7.org/fhir/ValueSet/ucum-bodylength')</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[41].resource.ofType(Observation), Line 3448, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[41].resource.ofType(Observation).meta.tag[0], Line 3409, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[41].resource.ofType(Observation).code.coding[0], Line 3429, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#20454-5 - Code can not be found in CodeSystem for 'http://loinc.org#20454-5'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[41].resource.ofType(Observation).subject, Line 3445, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[41].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 3450, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#260385009 - Code can not be found in CodeSystem for 'http://snomed.info/sct#260385009'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[41].resource.ofType(Observation), Line 3448, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[41].resource.ofType(Observation).code, Line 3428, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#20454-5)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[41].resource.ofType(Observation).subject, Line 3445, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[41].resource.ofType(Observation).subject, Line 3445, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[42].resource.ofType(Observation), Line 3535, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[42].resource.ofType(Observation).meta.tag[0], Line 3485, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[42].resource.ofType(Observation).code.coding[0], Line 3505, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#20405-7 - Code can not be found in CodeSystem for 'http://loinc.org#20405-7'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[42].resource.ofType(Observation).subject, Line 3532, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[42].resource.ofType(Observation).value.ofType(Quantity), Line 3536, Col 32]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mg/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mg/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[42].resource.ofType(Observation).referenceRange[0].low, Line 3523, Col 38]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mg/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mg/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[42].resource.ofType(Observation).referenceRange[0].high, Line 3517, Col 38]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mg/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mg/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[42].resource.ofType(Observation), Line 3535, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[42].resource.ofType(Observation).code, Line 3504, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#20405-7)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[42].resource.ofType(Observation).subject, Line 3532, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[42].resource.ofType(Observation).subject, Line 3532, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[43].resource.ofType(Observation), Line 3628, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[43].resource.ofType(Observation).meta.tag[0], Line 3568, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[43].resource.ofType(Observation).code.coding[0], Line 3588, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#17861-6 - Code can not be found in CodeSystem for 'http://loinc.org#17861-6'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[43].resource.ofType(Observation).subject, Line 3625, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[43].resource.ofType(Observation).referenceRange[0].low, Line 3606, Col 38]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mg/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mg/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[43].resource.ofType(Observation).referenceRange[0].high, Line 3600, Col 38]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mg/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mg/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[43].resource.ofType(Observation), Line 3628, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[43].resource.ofType(Observation).code, Line 3587, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#17861-6)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[43].resource.ofType(Observation).subject, Line 3625, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[43].resource.ofType(Observation).subject, Line 3625, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[44].resource.ofType(Observation), Line 3691, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[44].resource.ofType(Observation).meta.tag[0], Line 3659, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[44].resource.ofType(Observation).code.coding[0], Line 3678, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#8601-7 - Code can not be found in CodeSystem for 'http://loinc.org#8601-7'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[44].resource.ofType(Observation).subject, Line 3692, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[44].resource.ofType(Observation), Line 3691, Col 25]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[44].resource.ofType(Observation).meta.profile[0], Line 3691, Col 25]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[45].resource.ofType(Observation), Line 3773, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[45].resource.ofType(Observation).meta.tag[0], Line 3723, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[45].resource.ofType(Observation).code.coding[0], Line 3743, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#5803-2 - Code can not be found in CodeSystem for 'http://loinc.org#5803-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[45].resource.ofType(Observation).subject, Line 3770, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[45].resource.ofType(Observation).value.ofType(Quantity), Line 3774, Col 31]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#[pH] - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#[pH]'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[45].resource.ofType(Observation).referenceRange[0].low, Line 3761, Col 37]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#[pH] - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#[pH]'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[45].resource.ofType(Observation).referenceRange[0].high, Line 3755, Col 37]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#[pH] - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#[pH]'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[45].resource.ofType(Observation), Line 3773, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[45].resource.ofType(Observation).code, Line 3742, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#5803-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[45].resource.ofType(Observation).subject, Line 3770, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[45].resource.ofType(Observation).subject, Line 3770, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[46].resource.ofType(Observation), Line 3841, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[46].resource.ofType(Observation).meta.tag[0], Line 3805, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[46].resource.ofType(Observation).code.coding[0], Line 3824, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#8302-2 - Code can not be found in CodeSystem for 'http://loinc.org#8302-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[46].resource.ofType(Observation).subject, Line 3838, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[46].resource.ofType(Observation).value.ofType(Quantity), Line 3842, Col 29]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#cm - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#cm'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[46].resource.ofType(Observation), Line 3841, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[46].resource.ofType(Observation).code, Line 3823, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#8302-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[46].resource.ofType(Observation).subject, Line 3838, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[46].resource.ofType(Observation).subject, Line 3838, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[46].resource.ofType(Observation).value.ofType(Quantity).code, Line 3842, Col 29]</td>
				<td><pre>The value provided ('cm') is not in the value set http://hl7.org/fhir/ValueSet/ucum-bodylength|4.0.1 (http://hl7.org/fhir/ValueSet/ucum-bodylength), and a code is required from this value set) (error message = Unknown code 'cm' for in-memory expansion of ValueSet 'http://hl7.org/fhir/ValueSet/ucum-bodylength')</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[47].resource.ofType(Observation), Line 3904, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[47].resource.ofType(Observation).meta.tag[0], Line 3874, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[47].resource.ofType(Observation).code.coding[0], Line 3894, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#11279-7 - Code can not be found in CodeSystem for 'http://loinc.org#11279-7'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[47].resource.ofType(Observation).subject, Line 3905, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[47].resource.ofType(Observation), Line 3904, Col 25]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[47].resource.ofType(Observation).code, Line 3893, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#11279-7)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[47].resource.ofType(Observation).subject, Line 3905, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[47].resource.ofType(Observation).subject, Line 3905, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[48].resource.ofType(Observation), Line 3996, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[48].resource.ofType(Observation).meta.tag[0], Line 3936, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[48].resource.ofType(Observation).code.coding[0], Line 3956, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#2339-0 - Code can not be found in CodeSystem for 'http://loinc.org#2339-0'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[48].resource.ofType(Observation).subject, Line 3993, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[48].resource.ofType(Observation).referenceRange[0].low, Line 3974, Col 38]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mg/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mg/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[48].resource.ofType(Observation).referenceRange[0].high, Line 3968, Col 38]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mg/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mg/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[48].resource.ofType(Observation), Line 3996, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[48].resource.ofType(Observation).code, Line 3955, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#2339-0)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[48].resource.ofType(Observation).subject, Line 3993, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[48].resource.ofType(Observation).subject, Line 3993, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[49].resource.ofType(Observation), Line 4059, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[49].resource.ofType(Observation).meta.tag[0], Line 4027, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[49].resource.ofType(Observation).code.coding[0], Line 4046, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#8884-9 - Code can not be found in CodeSystem for 'http://loinc.org#8884-9'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[49].resource.ofType(Observation).subject, Line 4060, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[49].resource.ofType(Observation), Line 4059, Col 25]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[49].resource.ofType(Observation).meta.profile[0], Line 4059, Col 25]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[50].resource.ofType(Observation), Line 4130, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[50].resource.ofType(Observation).meta.tag[0], Line 4091, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[50].resource.ofType(Observation).code.coding[0], Line 4111, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#5802-4 - Code can not be found in CodeSystem for 'http://loinc.org#5802-4'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[50].resource.ofType(Observation).subject, Line 4127, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[50].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 4132, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#10828004 - Code can not be found in CodeSystem for 'http://snomed.info/sct#10828004'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[50].resource.ofType(Observation), Line 4130, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[50].resource.ofType(Observation).code, Line 4110, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#5802-4)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[50].resource.ofType(Observation).subject, Line 4127, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[50].resource.ofType(Observation).subject, Line 4127, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[51].resource.ofType(Observation), Line 4202, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[51].resource.ofType(Observation).meta.tag[0], Line 4166, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[51].resource.ofType(Observation).category[0], Line 4172, Col 17]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-category (http://hl7.org/fhir/ValueSet/observation-category), and a coding is recommended to come from this value set) (codes = http://loinc.org#LG41762-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[51].resource.ofType(Observation).category[0].coding[0], Line 4174, Col 23]</td>
				<td><pre>Unable to validate code http://loinc.org#LG41762-2 - Code can not be found in CodeSystem for 'http://loinc.org#LG41762-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[51].resource.ofType(Observation).code.coding[0], Line 4193, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#160695008 - Code can not be found in CodeSystem for 'http://snomed.info/sct#160695008'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[51].resource.ofType(Observation).subject, Line 4204, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[51].resource.ofType(Observation), Line 4202, Col 25]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[51].resource.ofType(Observation).meta.profile[0], Line 4202, Col 25]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-social-history-assessment' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[52].resource.ofType(Observation), Line 4272, Col 28]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[52].resource.ofType(Observation).meta.tag[0], Line 4233, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[52].resource.ofType(Observation).code.coding[0], Line 4253, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#5787-7 - Code can not be found in CodeSystem for 'http://loinc.org#5787-7'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[52].resource.ofType(Observation).subject, Line 4269, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[52].resource.ofType(Observation).value.ofType(Range).low, Line 4280, Col 36]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#/[HPF] - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#/[HPF]'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[52].resource.ofType(Observation).value.ofType(Range).high, Line 4274, Col 36]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#/[HPF] - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#/[HPF]'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[52].resource.ofType(Observation), Line 4272, Col 28]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[52].resource.ofType(Observation).code, Line 4252, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#5787-7)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[52].resource.ofType(Observation).subject, Line 4269, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[52].resource.ofType(Observation).subject, Line 4269, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[53].resource.ofType(Observation), Line 4344, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[53].resource.ofType(Observation).meta.tag[0], Line 4312, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[53].resource.ofType(Observation).code.coding[0], Line 4331, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#9843-4 - Code can not be found in CodeSystem for 'http://loinc.org#9843-4'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[53].resource.ofType(Observation).subject, Line 4343, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-2 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[53].resource.ofType(Observation).value.ofType(Quantity), Line 4345, Col 29]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#cm - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#cm'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[53].resource.ofType(Observation), Line 4344, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[53].resource.ofType(Observation).code, Line 4330, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#9843-4)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[53].resource.ofType(Observation).subject, Line 4343, Col 14]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-2 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[53].resource.ofType(Observation).subject, Line 4343, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-2 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[53].resource.ofType(Observation).value.ofType(Quantity).code, Line 4345, Col 29]</td>
				<td><pre>The value provided ('cm') is not in the value set http://hl7.org/fhir/ValueSet/ucum-bodylength|4.0.1 (http://hl7.org/fhir/ValueSet/ucum-bodylength), and a code is required from this value set) (error message = Unknown code 'cm' for in-memory expansion of ValueSet 'http://hl7.org/fhir/ValueSet/ucum-bodylength')</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[54].resource.ofType(Observation), Line 4412, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[54].resource.ofType(Observation).meta.tag[0], Line 4376, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[54].resource.ofType(Observation).code.coding[0], Line 4395, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#8867-4 - Code can not be found in CodeSystem for 'http://loinc.org#8867-4'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[54].resource.ofType(Observation).subject, Line 4409, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[54].resource.ofType(Observation).value.ofType(Quantity), Line 4413, Col 31]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#/min - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#/min'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[54].resource.ofType(Observation), Line 4412, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[54].resource.ofType(Observation).code, Line 4394, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#8867-4)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[54].resource.ofType(Observation).subject, Line 4409, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[54].resource.ofType(Observation).subject, Line 4409, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[55].resource.ofType(Observation), Line 4484, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[55].resource.ofType(Observation).meta.tag[0], Line 4445, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[55].resource.ofType(Observation).code.coding[0], Line 4465, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#32167-9 - Code can not be found in CodeSystem for 'http://loinc.org#32167-9'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[55].resource.ofType(Observation).subject, Line 4481, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[55].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 4486, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#81858005 - Code can not be found in CodeSystem for 'http://snomed.info/sct#81858005'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[55].resource.ofType(Observation), Line 4484, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[55].resource.ofType(Observation).code, Line 4464, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#32167-9)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[55].resource.ofType(Observation).subject, Line 4481, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[55].resource.ofType(Observation).subject, Line 4481, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[56].resource.ofType(Observation), Line 4556, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[56].resource.ofType(Observation).meta.tag[0], Line 4520, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[56].resource.ofType(Observation).code.coding[0], Line 4539, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#80948-3 - Code can not be found in CodeSystem for 'http://loinc.org#80948-3'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[56].resource.ofType(Observation).subject, Line 4553, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[56].resource.ofType(Observation).value.ofType(Quantity), Line 4557, Col 36]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#{T-Score} - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#{T-Score}'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[56].resource.ofType(Observation).value.ofType(Quantity), Line 4557, Col 36]</td>
				<td><pre>UCUM Codes that contain human readable annotations like {T-Score} can be misleading. Best Practice is not to use annotations in the UCUM code, and rather to make sure that Quantity.unit is correctly human readable </pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[56].resource.ofType(Observation), Line 4556, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[56].resource.ofType(Observation).meta.profile[0], Line 4556, Col 31]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-imaging' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[57].resource.ofType(Observation), Line 4624, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[57].resource.ofType(Observation).meta.tag[0], Line 4588, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[57].resource.ofType(Observation).code.coding[0], Line 4607, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#44976-9 - Code can not be found in CodeSystem for 'http://loinc.org#44976-9'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[57].resource.ofType(Observation).subject, Line 4621, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[57].resource.ofType(Observation).value.ofType(Quantity), Line 4625, Col 28]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#s - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#s'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[57].resource.ofType(Observation), Line 4624, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[57].resource.ofType(Observation).meta.profile[0], Line 4624, Col 31]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[58].resource.ofType(Observation), Line 4692, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[58].resource.ofType(Observation).meta.tag[0], Line 4656, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[58].resource.ofType(Observation).code.coding[0], Line 4675, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#80954-1 - Code can not be found in CodeSystem for 'http://loinc.org#80954-1'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[58].resource.ofType(Observation).subject, Line 4689, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[58].resource.ofType(Observation).value.ofType(Quantity), Line 4693, Col 32]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#g/cm2 - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#g/cm2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[58].resource.ofType(Observation), Line 4692, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[58].resource.ofType(Observation).meta.profile[0], Line 4692, Col 31]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-imaging' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[59].resource.ofType(Observation), Line 4760, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[59].resource.ofType(Observation).meta.tag[0], Line 4724, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[59].resource.ofType(Observation).code.coding[0], Line 4743, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#44975-1 - Code can not be found in CodeSystem for 'http://loinc.org#44975-1'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[59].resource.ofType(Observation).subject, Line 4757, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[59].resource.ofType(Observation).value.ofType(Quantity), Line 4761, Col 28]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#s - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#s'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[59].resource.ofType(Observation), Line 4760, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[59].resource.ofType(Observation).meta.profile[0], Line 4760, Col 31]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[60].resource.ofType(Observation), Line 4828, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[60].resource.ofType(Observation).meta.tag[0], Line 4792, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[60].resource.ofType(Observation).code.coding[0], Line 4811, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#39156-5 - Code can not be found in CodeSystem for 'http://loinc.org#39156-5'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[60].resource.ofType(Observation).subject, Line 4825, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[60].resource.ofType(Observation).value.ofType(Quantity), Line 4829, Col 32]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#kg/m2 - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#kg/m2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[60].resource.ofType(Observation), Line 4828, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[60].resource.ofType(Observation).code, Line 4810, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#39156-5)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[60].resource.ofType(Observation).subject, Line 4825, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[60].resource.ofType(Observation).subject, Line 4825, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[61].resource.ofType(Observation), Line 4921, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[61].resource.ofType(Observation).meta.tag[0], Line 4861, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[61].resource.ofType(Observation).code.coding[0], Line 4881, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#2075-0 - Code can not be found in CodeSystem for 'http://loinc.org#2075-0'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[61].resource.ofType(Observation).subject, Line 4918, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[61].resource.ofType(Observation).referenceRange[0].low, Line 4899, Col 39]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mmol/L - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mmol/L'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[61].resource.ofType(Observation).referenceRange[0].high, Line 4893, Col 39]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mmol/L - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mmol/L'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[61].resource.ofType(Observation), Line 4921, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[61].resource.ofType(Observation).code, Line 4880, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#2075-0)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[61].resource.ofType(Observation).subject, Line 4918, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[61].resource.ofType(Observation).subject, Line 4918, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[62].resource.ofType(Observation), Line 5013, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[62].resource.ofType(Observation).meta.tag[0], Line 4953, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[62].resource.ofType(Observation).code.coding[0], Line 4973, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#718-7 - Code can not be found in CodeSystem for 'http://loinc.org#718-7'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[62].resource.ofType(Observation).subject, Line 5010, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[62].resource.ofType(Observation).referenceRange[0].low, Line 4991, Col 37]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#g/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#g/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[62].resource.ofType(Observation).referenceRange[0].high, Line 4985, Col 37]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#g/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#g/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[62].resource.ofType(Observation), Line 5013, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[62].resource.ofType(Observation).code, Line 4972, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#718-7)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[62].resource.ofType(Observation).subject, Line 5010, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[62].resource.ofType(Observation).subject, Line 5010, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[63].resource.ofType(Observation), Line 5080, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[63].resource.ofType(Observation).meta.tag[0], Line 5044, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[63].resource.ofType(Observation).code.coding[0], Line 5063, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#44963-7 - Code can not be found in CodeSystem for 'http://loinc.org#44963-7'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[63].resource.ofType(Observation).subject, Line 5077, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[63].resource.ofType(Observation).value.ofType(Quantity), Line 5081, Col 28]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#s - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#s'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[63].resource.ofType(Observation), Line 5080, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[63].resource.ofType(Observation).meta.profile[0], Line 5080, Col 31]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-clinical-test' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[64].resource.ofType(Observation), Line 5186, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[64].resource.ofType(Observation).meta.tag[0], Line 5112, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[64].resource.ofType(Observation).code.coding[0], Line 5131, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#85354-9 - Code can not be found in CodeSystem for 'http://loinc.org#85354-9'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[64].resource.ofType(Observation).subject, Line 5187, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[64].resource.ofType(Observation).encounter, Line 5184, Col 14]</td>
				<td><pre>Details for Encounter/202201uscore-encounter-example-2 matching against Profile http://hl7.org/fhir/StructureDefinition/Encounter</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[64].resource.ofType(Observation).component[0].code.coding[0], Line 5143, Col 26]</td>
				<td><pre>Unable to validate code http://loinc.org#8480-6 - Code can not be found in CodeSystem for 'http://loinc.org#8480-6'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[64].resource.ofType(Observation).component[0].value.ofType(Quantity), Line 5152, Col 39]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mm[Hg] - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mm[Hg]'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[64].resource.ofType(Observation).component[1].code.coding[0], Line 5161, Col 26]</td>
				<td><pre>Unable to validate code http://loinc.org#8462-4 - Code can not be found in CodeSystem for 'http://loinc.org#8462-4'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[64].resource.ofType(Observation), Line 5186, Col 25]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[64].resource.ofType(Observation).code, Line 5130, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#85354-9)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[64].resource.ofType(Observation).subject, Line 5187, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[64].resource.ofType(Observation).subject, Line 5187, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[64].resource.ofType(Observation).component[0].code, Line 5142, Col 33]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#8480-6)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[64].resource.ofType(Observation).component[1].code, Line 5160, Col 33]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#8462-4)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[65].resource.ofType(Observation), Line 5265, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[65].resource.ofType(Observation).meta.tag[0], Line 5217, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[65].resource.ofType(Observation).code.coding[0], Line 5237, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#5811-5 - Code can not be found in CodeSystem for 'http://loinc.org#5811-5'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[65].resource.ofType(Observation).subject, Line 5262, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[65].resource.ofType(Observation).value.ofType(Quantity), Line 5266, Col 51]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#{urine specific gravity} - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#{urine specific gravity}'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[65].resource.ofType(Observation).value.ofType(Quantity), Line 5266, Col 51]</td>
				<td><pre>UCUM Codes that contain human readable annotations like {urine specific gravity} can be misleading. Best Practice is not to use annotations in the UCUM code, and rather to make sure that Quantity.unit is correctly human readable </pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[65].resource.ofType(Observation).referenceRange[0].low, Line 5254, Col 57]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#{urine specific gravity} - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#{urine specific gravity}'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[65].resource.ofType(Observation).referenceRange[0].low, Line 5254, Col 57]</td>
				<td><pre>UCUM Codes that contain human readable annotations like {urine specific gravity} can be misleading. Best Practice is not to use annotations in the UCUM code, and rather to make sure that Quantity.unit is correctly human readable </pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[65].resource.ofType(Observation).referenceRange[0].high, Line 5249, Col 57]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#{urine specific gravity} - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#{urine specific gravity}'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[65].resource.ofType(Observation).referenceRange[0].high, Line 5249, Col 57]</td>
				<td><pre>UCUM Codes that contain human readable annotations like {urine specific gravity} can be misleading. Best Practice is not to use annotations in the UCUM code, and rather to make sure that Quantity.unit is correctly human readable </pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[65].resource.ofType(Observation), Line 5265, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[65].resource.ofType(Observation).code, Line 5236, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#5811-5)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[65].resource.ofType(Observation).subject, Line 5262, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[65].resource.ofType(Observation).subject, Line 5262, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[66].resource.ofType(Observation), Line 5332, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[66].resource.ofType(Observation).meta.tag[0], Line 5296, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[66].resource.ofType(Observation).code.coding[0], Line 5315, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#80946-7 - Code can not be found in CodeSystem for 'http://loinc.org#80946-7'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[66].resource.ofType(Observation).subject, Line 5329, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[66].resource.ofType(Observation).value.ofType(Quantity), Line 5333, Col 36]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#{T-Score} - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#{T-Score}'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[66].resource.ofType(Observation).value.ofType(Quantity), Line 5333, Col 36]</td>
				<td><pre>UCUM Codes that contain human readable annotations like {T-Score} can be misleading. Best Practice is not to use annotations in the UCUM code, and rather to make sure that Quantity.unit is correctly human readable </pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[66].resource.ofType(Observation), Line 5332, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[66].resource.ofType(Observation).meta.profile[0], Line 5332, Col 31]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-imaging' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[67].resource.ofType(Observation), Line 5400, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[67].resource.ofType(Observation).meta.tag[0], Line 5364, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[67].resource.ofType(Observation).code.coding[0], Line 5383, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#80940-0 - Code can not be found in CodeSystem for 'http://loinc.org#80940-0'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[67].resource.ofType(Observation).subject, Line 5397, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[67].resource.ofType(Observation).value.ofType(Quantity), Line 5401, Col 36]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#{Z-Score} - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#{Z-Score}'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[67].resource.ofType(Observation).value.ofType(Quantity), Line 5401, Col 36]</td>
				<td><pre>UCUM Codes that contain human readable annotations like {Z-Score} can be misleading. Best Practice is not to use annotations in the UCUM code, and rather to make sure that Quantity.unit is correctly human readable </pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[67].resource.ofType(Observation), Line 5400, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[67].resource.ofType(Observation).meta.profile[0], Line 5400, Col 31]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-imaging' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[68].resource.ofType(Observation), Line 5472, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[68].resource.ofType(Observation).meta.tag[0], Line 5433, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[68].resource.ofType(Observation).code.coding[0], Line 5453, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#2514-8 - Code can not be found in CodeSystem for 'http://loinc.org#2514-8'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[68].resource.ofType(Observation).subject, Line 5469, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[68].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 5474, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#260385009 - Code can not be found in CodeSystem for 'http://snomed.info/sct#260385009'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[68].resource.ofType(Observation), Line 5472, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[68].resource.ofType(Observation).code, Line 5452, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#2514-8)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[68].resource.ofType(Observation).subject, Line 5469, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[68].resource.ofType(Observation).subject, Line 5469, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[69].resource.ofType(Observation), Line 5569, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[69].resource.ofType(Observation).meta.tag[0], Line 5509, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[69].resource.ofType(Observation).code.coding[0], Line 5529, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#1975-2 - Code can not be found in CodeSystem for 'http://loinc.org#1975-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[69].resource.ofType(Observation).subject, Line 5566, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[69].resource.ofType(Observation).referenceRange[0].low, Line 5547, Col 38]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mg/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mg/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[69].resource.ofType(Observation).referenceRange[0].high, Line 5541, Col 38]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mg/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mg/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[69].resource.ofType(Observation), Line 5569, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[69].resource.ofType(Observation).code, Line 5528, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#1975-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[69].resource.ofType(Observation).subject, Line 5566, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[69].resource.ofType(Observation).subject, Line 5566, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[70].resource.ofType(Observation), Line 5640, Col 28]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[70].resource.ofType(Observation).meta.tag[0], Line 5601, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[70].resource.ofType(Observation).code.coding[0], Line 5621, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#5821-4 - Code can not be found in CodeSystem for 'http://loinc.org#5821-4'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[70].resource.ofType(Observation).subject, Line 5637, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[70].resource.ofType(Observation).value.ofType(Range).low, Line 5648, Col 36]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#/[HPF] - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#/[HPF]'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[70].resource.ofType(Observation).value.ofType(Range).high, Line 5642, Col 36]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#/[HPF] - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#/[HPF]'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[70].resource.ofType(Observation), Line 5640, Col 28]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[70].resource.ofType(Observation).code, Line 5620, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#5821-4)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[70].resource.ofType(Observation).subject, Line 5637, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[70].resource.ofType(Observation).subject, Line 5637, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[71].resource.ofType(Observation), Line 5716, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[71].resource.ofType(Observation).meta.tag[0], Line 5680, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[71].resource.ofType(Observation).code.coding[0], Line 5699, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#29463-7 - Code can not be found in CodeSystem for 'http://loinc.org#29463-7'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[71].resource.ofType(Observation).subject, Line 5713, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[71].resource.ofType(Observation).value.ofType(Quantity), Line 5717, Col 29]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#kg - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#kg'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[71].resource.ofType(Observation), Line 5716, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[71].resource.ofType(Observation).code, Line 5698, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#29463-7)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[71].resource.ofType(Observation).subject, Line 5713, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[71].resource.ofType(Observation).subject, Line 5713, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[71].resource.ofType(Observation).value.ofType(Quantity).code, Line 5717, Col 29]</td>
				<td><pre>The value provided ('kg') is not in the value set http://hl7.org/fhir/ValueSet/ucum-bodyweight|4.0.1 (http://hl7.org/fhir/ValueSet/ucum-bodyweight), and a code is required from this value set) (error message = Unknown code 'kg' for in-memory expansion of ValueSet 'http://hl7.org/fhir/ValueSet/ucum-bodyweight')</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[72].resource.ofType(Observation), Line 5789, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[72].resource.ofType(Observation).meta.tag[0], Line 5748, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[72].resource.ofType(Observation).code.coding[0], Line 5767, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#59576-9 - Code can not be found in CodeSystem for 'http://loinc.org#59576-9'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[72].resource.ofType(Observation).subject, Line 5786, Col 43]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-3 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[72].resource.ofType(Observation).value.ofType(Quantity), Line 5790, Col 28]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#% - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#%'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[72].resource.ofType(Observation), Line 5789, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[72].resource.ofType(Observation).code, Line 5766, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs (http://hl7.org/fhir/us/core/ValueSet/us-core-vital-signs), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#59576-9)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[72].resource.ofType(Observation).subject, Line 5786, Col 43]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-3 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[72].resource.ofType(Observation).subject, Line 5786, Col 43]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-3 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[73].resource.ofType(Observation), Line 5882, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[73].resource.ofType(Observation).meta.tag[0], Line 5822, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[73].resource.ofType(Observation).code.coding[0], Line 5842, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#789-8 - Code can not be found in CodeSystem for 'http://loinc.org#789-8'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[73].resource.ofType(Observation).subject, Line 5879, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[73].resource.ofType(Observation).referenceRange[0].low, Line 5860, Col 40]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#10*6/uL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#10*6/uL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[73].resource.ofType(Observation).referenceRange[0].high, Line 5854, Col 40]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#10*6/uL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#10*6/uL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[73].resource.ofType(Observation), Line 5882, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[73].resource.ofType(Observation).code, Line 5841, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#789-8)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[73].resource.ofType(Observation).subject, Line 5879, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[73].resource.ofType(Observation).subject, Line 5879, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[74].resource.ofType(Observation), Line 5974, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[74].resource.ofType(Observation).meta.tag[0], Line 5914, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[74].resource.ofType(Observation).code.coding[0], Line 5934, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#3094-0 - Code can not be found in CodeSystem for 'http://loinc.org#3094-0'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[74].resource.ofType(Observation).subject, Line 5971, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[74].resource.ofType(Observation).referenceRange[0].low, Line 5952, Col 38]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mg/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mg/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[74].resource.ofType(Observation).referenceRange[0].high, Line 5946, Col 38]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mg/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mg/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[74].resource.ofType(Observation), Line 5974, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[74].resource.ofType(Observation).code, Line 5933, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#3094-0)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[74].resource.ofType(Observation).subject, Line 5971, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[74].resource.ofType(Observation).subject, Line 5971, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[75].resource.ofType(Observation), Line 6066, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[75].resource.ofType(Observation).meta.tag[0], Line 6006, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[75].resource.ofType(Observation).code.coding[0], Line 6026, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#2028-9 - Code can not be found in CodeSystem for 'http://loinc.org#2028-9'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[75].resource.ofType(Observation).subject, Line 6063, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[75].resource.ofType(Observation).referenceRange[0].low, Line 6044, Col 39]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mmol/L - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mmol/L'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[75].resource.ofType(Observation).referenceRange[0].high, Line 6038, Col 39]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mmol/L - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mmol/L'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[75].resource.ofType(Observation), Line 6066, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[75].resource.ofType(Observation).code, Line 6025, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#2028-9)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[75].resource.ofType(Observation).subject, Line 6063, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[75].resource.ofType(Observation).subject, Line 6063, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[76].resource.ofType(Observation), Line 6137, Col 38]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[76].resource.ofType(Observation).meta.tag[0], Line 6098, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[76].resource.ofType(Observation).code.coding[0], Line 6118, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#25145-4 - Code can not be found in CodeSystem for 'http://loinc.org#25145-4'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[76].resource.ofType(Observation).subject, Line 6134, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[76].resource.ofType(Observation).value.ofType(CodeableConcept).coding[0], Line 6139, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#260350009 - Code can not be found in CodeSystem for 'http://snomed.info/sct#260350009'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[76].resource.ofType(Observation), Line 6137, Col 38]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[76].resource.ofType(Observation).code, Line 6117, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#25145-4)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[76].resource.ofType(Observation).subject, Line 6134, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[76].resource.ofType(Observation).subject, Line 6134, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[77].resource.ofType(Observation), Line 6209, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[77].resource.ofType(Observation).meta.tag[0], Line 6173, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[77].resource.ofType(Observation).code.coding[0], Line 6192, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#80938-4 - Code can not be found in CodeSystem for 'http://loinc.org#80938-4'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[77].resource.ofType(Observation).subject, Line 6206, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[77].resource.ofType(Observation).value.ofType(Quantity), Line 6210, Col 36]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#{Z-Score} - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#{Z-Score}'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[77].resource.ofType(Observation).value.ofType(Quantity), Line 6210, Col 36]</td>
				<td><pre>UCUM Codes that contain human readable annotations like {Z-Score} can be misleading. Best Practice is not to use annotations in the UCUM code, and rather to make sure that Quantity.unit is correctly human readable </pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[77].resource.ofType(Observation), Line 6209, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[77].resource.ofType(Observation).meta.profile[0], Line 6209, Col 31]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-imaging' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[78].resource.ofType(Observation), Line 6302, Col 31]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[78].resource.ofType(Observation).meta.tag[0], Line 6242, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[78].resource.ofType(Observation).code.coding[0], Line 6262, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#786-4 - Code can not be found in CodeSystem for 'http://loinc.org#786-4'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[78].resource.ofType(Observation).subject, Line 6299, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[78].resource.ofType(Observation).referenceRange[0].low, Line 6280, Col 37]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#g/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#g/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[78].resource.ofType(Observation).referenceRange[0].high, Line 6274, Col 37]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#g/dL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#g/dL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[78].resource.ofType(Observation), Line 6302, Col 31]</td>
				<td><pre>All observations should have a performer</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[78].resource.ofType(Observation).code, Line 6261, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/observation-codes (http://hl7.org/fhir/ValueSet/observation-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#786-4)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[78].resource.ofType(Observation).subject, Line 6299, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[78].resource.ofType(Observation).subject, Line 6299, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[79].resource.ofType(Media), Line 6346, Col 22]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[79].resource.ofType(Media).meta.tag[0], Line 6330, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[79].resource.ofType(Media).subject, Line 6345, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[80].resource.ofType(Medication), Line 6386, Col 22]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[80].resource.ofType(Medication).meta.tag[0], Line 6381, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[80].resource.ofType(Medication).code, Line 6387, Col 27]</td>
				<td><pre>ValueSet http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1010.4 not found by validator</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[81].resource.ofType(Medication), Line 6427, Col 22]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[81].resource.ofType(Medication).meta.tag[0], Line 6422, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[81].resource.ofType(Medication).code, Line 6428, Col 27]</td>
				<td><pre>ValueSet http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1010.4 not found by validator</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[82].resource.ofType(RelatedPerson), Line 6490, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[82].resource.ofType(RelatedPerson).meta.tag[0], Line 6463, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[82].resource.ofType(RelatedPerson).patient, Line 6491, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[82].resource.ofType(RelatedPerson).meta.profile[0], Line 6490, Col 25]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-relatedperson' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[83].resource.ofType(MedicationRequest), Line 6602, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[83].resource.ofType(MedicationRequest).meta.tag[0], Line 6543, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[83].resource.ofType(MedicationRequest).contained[0].ofType(Medication), Line 6549, Col 17]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[83].resource.ofType(MedicationRequest).subject, Line 6603, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[83].resource.ofType(MedicationRequest).dispenseRequest.quantity, Line 6574, Col 32]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[83].resource.ofType(MedicationRequest).subject, Line 6603, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[83].resource.ofType(MedicationRequest).subject, Line 6603, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[84].resource.ofType(ServiceRequest), Line 6665, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[84].resource.ofType(ServiceRequest).meta.tag[0], Line 6632, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[84].resource.ofType(ServiceRequest).category[0].coding[0], Line 6641, Col 23]</td>
				<td><pre>Unable to validate code http://loinc.org#LG41762-2 - Code can not be found in CodeSystem for 'http://loinc.org#LG41762-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[84].resource.ofType(ServiceRequest).code.coding[0], Line 6652, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#467771000124109 - Code can not be found in CodeSystem for 'http://snomed.info/sct#467771000124109'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[84].resource.ofType(ServiceRequest).subject, Line 6667, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[84].resource.ofType(ServiceRequest).meta.profile[0], Line 6665, Col 25]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-servicerequest' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[85].resource.ofType(Goal), Line 6710, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[85].resource.ofType(Goal).meta.tag[0], Line 6694, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[85].resource.ofType(Goal).description.coding[0], Line 6701, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#406156006 - Code can not be found in CodeSystem for 'http://snomed.info/sct#406156006'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[85].resource.ofType(Goal).subject, Line 6711, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[85].resource.ofType(Goal).subject, Line 6711, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[85].resource.ofType(Goal).subject, Line 6711, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[86].resource.ofType(Procedure), Line 6767, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[86].resource.ofType(Procedure).meta.tag[0], Line 6745, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[86].resource.ofType(Procedure).code.coding[0], Line 6752, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#24165007 - Code can not be found in CodeSystem for 'http://snomed.info/sct#24165007'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[86].resource.ofType(Procedure).subject, Line 6768, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[86].resource.ofType(Procedure).code, Line 6751, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-procedure-code (http://hl7.org/fhir/us/core/ValueSet/us-core-procedure-code), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://snomed.info/sct#24165007, http://www.cms.gov/Medicare/Coding/ICD10#HZ30ZZZ)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[86].resource.ofType(Procedure).subject, Line 6768, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[86].resource.ofType(Procedure).subject, Line 6768, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[87].resource.ofType(AllergyIntolerance), Line 6845, Col 36]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[87].resource.ofType(AllergyIntolerance).meta.tag[0], Line 6797, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[87].resource.ofType(AllergyIntolerance).code.coding[0], Line 6815, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#763875007 - Code can not be found in CodeSystem for 'http://snomed.info/sct#763875007'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[87].resource.ofType(AllergyIntolerance).patient, Line 6825, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[87].resource.ofType(AllergyIntolerance).reaction[0].manifestation[0].coding[0], Line 6833, Col 29]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#271807003 - Code can not be found in CodeSystem for 'http://snomed.info/sct#271807003'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[87].resource.ofType(AllergyIntolerance).code, Line 6814, Col 27]</td>
				<td><pre>ValueSet http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1186.8 not found by validator</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[87].resource.ofType(AllergyIntolerance).patient, Line 6825, Col 41]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[87].resource.ofType(AllergyIntolerance).patient, Line 6825, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[87].resource.ofType(AllergyIntolerance).reaction[0].manifestation[0], Line 6831, Col 23]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/clinical-findings (http://hl7.org/fhir/ValueSet/clinical-findings), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://snomed.info/sct#271807003)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam), Line 6956, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).meta.tag[0], Line 6879, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).subject, Line 6957, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).participant[0].role[0].coding[0], Line 6894, Col 29]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#17561000 - Code can not be found in CodeSystem for 'http://snomed.info/sct#17561000'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).participant[1].role[0].coding[0], Line 6911, Col 29]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#453231000124104 - Code can not be found in CodeSystem for 'http://snomed.info/sct#453231000124104'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).participant[2].role[0].coding[0], Line 6928, Col 29]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#116154003 - Code can not be found in CodeSystem for 'http://snomed.info/sct#116154003'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).participant[2].member, Line 6922, Col 47]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).participant[3].role[0].coding[0], Line 6945, Col 29]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#34581001 - Code can not be found in CodeSystem for 'http://snomed.info/sct#34581001'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).subject, Line 6957, Col 41]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).subject, Line 6957, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).participant[0].role[0], Line 6892, Col 23]</td>
				<td><pre>ValueSet http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1099.30 not found by validator</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).participant[1].role[0], Line 6909, Col 23]</td>
				<td><pre>ValueSet http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1099.30 not found by validator</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).participant[2].role[0], Line 6926, Col 23]</td>
				<td><pre>ValueSet http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1099.30 not found by validator</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).participant[2].member, Line 6922, Col 47]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner, http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization, http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient, http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitionerrole, http://hl7.org/fhir/us/core/StructureDefinition/us-core-careteam, http://hl7.org/fhir/StructureDefinition/RelatedPerson</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).participant[2].member, Line 6922, Col 47]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[88].resource.ofType(CareTeam).participant[3].role[0], Line 6943, Col 23]</td>
				<td><pre>ValueSet http://cts.nlm.nih.gov/fhir/ValueSet/2.16.840.1.113762.1.4.1099.30 not found by validator</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport), Line 7041, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).meta.tag[0], Line 6986, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).category[0].coding[0], Line 6994, Col 23]</td>
				<td><pre>Unable to validate code http://loinc.org#LP29684-5 - Code can not be found in CodeSystem for 'http://loinc.org#LP29684-5'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).code, Line 7004, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/report-codes (http://hl7.org/fhir/ValueSet/report-codes), and a coding is recommended to come from this value set) (codes = http://loinc.org#83311-1)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).code.coding[0], Line 7005, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#83311-1 - Code can not be found in CodeSystem for 'http://loinc.org#83311-1'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).subject, Line 7043, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).result[0], Line 7021, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-65 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).result[1], Line 7024, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-54 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).result[2], Line 7027, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-23 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).result[3], Line 7030, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-55 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).result[4], Line 7033, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-44 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).result[5], Line 7036, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-46 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).category[0], Line 6992, Col 17]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-category (http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-category), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#LP29684-5)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).code, Line 7004, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-report-and-note-codes (http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-report-and-note-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#83311-1)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).subject, Line 7043, Col 14]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[89].resource.ofType(DiagnosticReport).subject, Line 7043, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport), Line 7142, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).meta.tag[0], Line 7070, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).code, Line 7087, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/report-codes (http://hl7.org/fhir/ValueSet/report-codes), and a coding is recommended to come from this value set) (codes = http://loinc.org#24323-8)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).code.coding[0], Line 7088, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#24323-8 - Code can not be found in CodeSystem for 'http://loinc.org#24323-8'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).subject, Line 7143, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[0], Line 7104, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-57 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[1], Line 7108, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-18 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[2], Line 7112, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-16 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[3], Line 7116, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-19 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[4], Line 7120, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-63 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[5], Line 7124, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-49 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[6], Line 7128, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-31 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[7], Line 7132, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-62 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[8], Line 7136, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-36 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).code, Line 7087, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-lab-codes (http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-lab-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#24323-8)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).subject, Line 7143, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).subject, Line 7143, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[0], Line 7104, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-57 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[0], Line 7104, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-57 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[1], Line 7108, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-18 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[1], Line 7108, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-18 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[2], Line 7112, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-16 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[2], Line 7112, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-16 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[3], Line 7116, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-19 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[3], Line 7116, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-19 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[4], Line 7120, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-63 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[4], Line 7120, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-63 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[5], Line 7124, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-49 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[5], Line 7124, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-49 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[6], Line 7128, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-31 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[6], Line 7128, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-31 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[7], Line 7132, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-62 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[7], Line 7132, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-62 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[8], Line 7136, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-36 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[90].resource.ofType(DiagnosticReport).result[8], Line 7136, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-36 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[91].resource.ofType(DocumentReference), Line 7228, Col 22]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[91].resource.ofType(DocumentReference).meta.tag[0], Line 7172, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[91].resource.ofType(DocumentReference).type, Line 7229, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/c80-doc-typecodes (http://hl7.org/fhir/ValueSet/c80-doc-typecodes), and a coding is recommended to come from this value set) (codes = http://loinc.org#34133-9)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[91].resource.ofType(DocumentReference).type.coding[0], Line 7230, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#34133-9 - Code can not be found in CodeSystem for 'http://loinc.org#34133-9'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[91].resource.ofType(DocumentReference).subject, Line 7225, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[91].resource.ofType(DocumentReference).content[0].format, Line 7203, Col 73]</td>
				<td><pre>Could not confirm that the codes provided are in the value set http://hl7.org/fhir/ValueSet/formatcodes, and a code is recommended to come from this value set</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[91].resource.ofType(DocumentReference).type, Line 7229, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-documentreference-type (http://hl7.org/fhir/us/core/ValueSet/us-core-documentreference-type), and a coding from this value set is required) (codes = http://loinc.org#34133-9)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[91].resource.ofType(DocumentReference).subject, Line 7225, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[91].resource.ofType(DocumentReference).subject, Line 7225, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[91].resource.ofType(DocumentReference).content[0].format, Line 7203, Col 73]</td>
				<td><pre>Could not confirm that the codes provided are in the value set http://hl7.org/fhir/ValueSet/formatcodes, and a code should come from this value set unless it has no suitable code (the validator cannot judge what is suitable)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport), Line 7337, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).meta.tag[0], Line 7264, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).category[0].coding[0], Line 7272, Col 23]</td>
				<td><pre>Unable to validate code http://loinc.org#LP29708-2 - Code can not be found in CodeSystem for 'http://loinc.org#LP29708-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).code, Line 7282, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/report-codes (http://hl7.org/fhir/ValueSet/report-codes), and a coding is recommended to come from this value set) (codes = http://loinc.org#45033-8)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).code.coding[0], Line 7283, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#45033-8 - Code can not be found in CodeSystem for 'http://loinc.org#45033-8'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).subject, Line 7339, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).result[0], Line 7308, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-11 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).result[1], Line 7311, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-10 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).result[2], Line 7314, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-45 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).result[3], Line 7317, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-47 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).result[4], Line 7320, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-22 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).result[5], Line 7323, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-37 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).result[6], Line 7326, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-6 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).result[7], Line 7329, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-32 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).result[8], Line 7332, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-51 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).category[0], Line 7270, Col 17]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-category (http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-category), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#LP29708-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).code, Line 7282, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-report-and-note-codes (http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-report-and-note-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#45033-8)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).subject, Line 7339, Col 14]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[92].resource.ofType(DiagnosticReport).subject, Line 7339, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[93].resource.ofType(CarePlan), Line 7383, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[93].resource.ofType(CarePlan).meta.tag[0], Line 7366, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[93].resource.ofType(CarePlan).subject, Line 7384, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[93].resource.ofType(CarePlan), Line 7383, Col 25]</td>
				<td><pre>CarePlan.text: minimum required = 1, but only found 0 (from http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[93].resource.ofType(CarePlan).subject, Line 7384, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[93].resource.ofType(CarePlan).subject, Line 7384, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport), Line 7518, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).meta.tag[0], Line 7413, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).code, Line 7430, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/report-codes (http://hl7.org/fhir/ValueSet/report-codes), and a coding is recommended to come from this value set) (codes = http://loinc.org#24356-8)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).code.coding[0], Line 7431, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#24356-8 - Code can not be found in CodeSystem for 'http://loinc.org#24356-8'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).subject, Line 7519, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[0], Line 7448, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-12 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[1], Line 7452, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-43 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[2], Line 7456, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-9 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[3], Line 7460, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-2 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[4], Line 7464, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-56 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[5], Line 7468, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-53 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[6], Line 7472, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-33 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[7], Line 7476, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-29 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[8], Line 7480, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-30 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[9], Line 7484, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-38 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[10], Line 7488, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-27 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[11], Line 7492, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-3 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[12], Line 7496, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-35 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[13], Line 7500, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-64 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[14], Line 7504, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-40 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[15], Line 7508, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-58 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[16], Line 7512, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-8 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).code, Line 7430, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-lab-codes (http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-lab-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#24356-8)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).subject, Line 7519, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).subject, Line 7519, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[0], Line 7448, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-12 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[0], Line 7448, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-12 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[1], Line 7452, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-43 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[1], Line 7452, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-43 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[2], Line 7456, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-9 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[2], Line 7456, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-9 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[3], Line 7460, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-2 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[3], Line 7460, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-2 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[4], Line 7464, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-56 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[4], Line 7464, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-56 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[5], Line 7468, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-53 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[5], Line 7468, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-53 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[6], Line 7472, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-33 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[6], Line 7472, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-33 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[7], Line 7476, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-29 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[7], Line 7476, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-29 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[8], Line 7480, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-30 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[8], Line 7480, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-30 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[9], Line 7484, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-38 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[9], Line 7484, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-38 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[10], Line 7488, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-27 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[10], Line 7488, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-27 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[11], Line 7492, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-3 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[11], Line 7492, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-3 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[12], Line 7496, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-35 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[12], Line 7496, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-35 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[13], Line 7500, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-64 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[13], Line 7500, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-64 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[14], Line 7504, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-40 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[14], Line 7504, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-40 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[15], Line 7508, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-58 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[15], Line 7508, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-58 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[16], Line 7512, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-8 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[94].resource.ofType(DiagnosticReport).result[16], Line 7512, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-8 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[95].resource.ofType(Device), Line 7561, Col 22]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[95].resource.ofType(Device).meta.tag[0], Line 7548, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[95].resource.ofType(Device).type.coding[0], Line 7563, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#468063009 - Code can not be found in CodeSystem for 'http://snomed.info/sct#468063009'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[95].resource.ofType(Device).patient, Line 7556, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[95].resource.ofType(Device).type, Line 7562, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/device-kind (http://hl7.org/fhir/ValueSet/device-kind), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://snomed.info/sct#468063009)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[95].resource.ofType(Device).patient, Line 7556, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[95].resource.ofType(Device).patient, Line 7556, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[96].resource.ofType(Condition), Line 7650, Col 36]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[96].resource.ofType(Condition).meta.tag[0], Line 7602, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[96].resource.ofType(Condition).category[0], Line 7614, Col 17]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/condition-category (http://hl7.org/fhir/ValueSet/condition-category), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#LG41762-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[96].resource.ofType(Condition).category[0].coding[0], Line 7616, Col 23]</td>
				<td><pre>Unable to validate code http://loinc.org#LG41762-2 - Code can not be found in CodeSystem for 'http://loinc.org#LG41762-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[96].resource.ofType(Condition).code.coding[0], Line 7637, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#445281000124101 - Code can not be found in CodeSystem for 'http://snomed.info/sct#445281000124101'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[96].resource.ofType(Condition).subject, Line 7647, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[96].resource.ofType(Condition), Line 7650, Col 36]</td>
				<td><pre>us-core-1: 'A code in Condition.category SHOULD be from US Core Condition Category Codes value set.' Rule 'A code in Condition.category SHOULD be from US Core Condition Category Codes value set.' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[96].resource.ofType(Condition).category[0], Line 7614, Col 17]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-condition-category (http://hl7.org/fhir/us/core/ValueSet/us-core-condition-category), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#LG41762-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[96].resource.ofType(Condition).code, Line 7636, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-condition-code (http://hl7.org/fhir/us/core/ValueSet/us-core-condition-code), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://snomed.info/sct#445281000124101)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[96].resource.ofType(Condition).subject, Line 7647, Col 41]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[96].resource.ofType(Condition).subject, Line 7647, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[97].resource.ofType(Device), Line 7707, Col 22]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[97].resource.ofType(Device).meta.tag[0], Line 7686, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[97].resource.ofType(Device).type.coding[0], Line 7709, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#19257004 - Code can not be found in CodeSystem for 'http://snomed.info/sct#19257004'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[97].resource.ofType(Device).patient, Line 7702, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[97].resource.ofType(Device).type, Line 7708, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/device-kind (http://hl7.org/fhir/ValueSet/device-kind), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://snomed.info/sct#19257004)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[97].resource.ofType(Device).patient, Line 7702, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[97].resource.ofType(Device).patient, Line 7702, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[98].resource.ofType(Goal), Line 7759, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[98].resource.ofType(Goal).meta.tag[0], Line 7750, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[98].resource.ofType(Goal).subject, Line 7760, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[98].resource.ofType(Goal).subject, Line 7760, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[98].resource.ofType(Goal).subject, Line 7760, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport), Line 7846, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).meta.tag[0], Line 7794, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).code, Line 7811, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/report-codes (http://hl7.org/fhir/ValueSet/report-codes), and a coding is recommended to come from this value set) (codes = http://loinc.org#58410-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).code.coding[0], Line 7812, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#58410-2 - Code can not be found in CodeSystem for 'http://loinc.org#58410-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).subject, Line 7847, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).result[0], Line 7828, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-4 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).result[1], Line 7832, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-66 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).result[2], Line 7836, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-50 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).result[3], Line 7840, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-61 matching against Profile http://hl7.org/fhir/StructureDefinition/Observation</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).code, Line 7811, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-lab-codes (http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-lab-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#58410-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).subject, Line 7847, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).subject, Line 7847, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).result[0], Line 7828, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-4 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).result[0], Line 7828, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-4 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).result[1], Line 7832, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-66 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).result[1], Line 7832, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-66 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).result[2], Line 7836, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-50 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).result[2], Line 7836, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-50 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).result[3], Line 7840, Col 17]</td>
				<td><pre>Unable to find a match for profile Observation/202201uscore-observation-example-61 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[99].resource.ofType(DiagnosticReport).result[3], Line 7840, Col 17]</td>
				<td><pre>Details for Observation/202201uscore-observation-example-61 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-observation-lab</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[100].resource.ofType(MedicationRequest), Line 7915, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[100].resource.ofType(MedicationRequest).meta.tag[0], Line 7876, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[100].resource.ofType(MedicationRequest).subject, Line 7916, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[100].resource.ofType(MedicationRequest).encounter, Line 7888, Col 42]</td>
				<td><pre>Details for Encounter/202201uscore-encounter-example-3 matching against Profile http://hl7.org/fhir/StructureDefinition/Encounter</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[100].resource.ofType(MedicationRequest).requester, Line 7911, Col 49]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[100].resource.ofType(MedicationRequest).medication.ofType(CodeableConcept), Line 7900, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-medication-codes (http://hl7.org/fhir/us/core/ValueSet/us-core-medication-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://www.nlm.nih.gov/research/umls/rxnorm#1187314)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[100].resource.ofType(MedicationRequest).subject, Line 7916, Col 41]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[100].resource.ofType(MedicationRequest).subject, Line 7916, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[100].resource.ofType(MedicationRequest).requester, Line 7911, Col 49]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-practitioner, http://hl7.org/fhir/us/core/StructureDefinition/us-core-organization, http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[100].resource.ofType(MedicationRequest).requester, Line 7911, Col 49]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[101].resource.ofType(Procedure), Line 7984, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[101].resource.ofType(Procedure).meta.tag[0], Line 7945, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[101].resource.ofType(Procedure).subject, Line 7986, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[101].resource.ofType(Procedure).focalDevice[0].manipulated, Line 7963, Col 20]</td>
				<td><pre>Details for Device/202201uscore-device-example-2 matching against Profile http://hl7.org/fhir/StructureDefinition/Device</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[101].resource.ofType(Procedure).code, Line 7951, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-procedure-code (http://hl7.org/fhir/us/core/ValueSet/us-core-procedure-code), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://www.ama-assn.org/go/cpt#33249)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[101].resource.ofType(Procedure).subject, Line 7986, Col 14]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[101].resource.ofType(Procedure).subject, Line 7986, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[102].resource.ofType(Immunization), Line 8025, Col 29]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[102].resource.ofType(Immunization).meta.tag[0], Line 8013, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[102].resource.ofType(Immunization).patient, Line 8020, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[102].resource.ofType(Immunization).vaccineCode, Line 8026, Col 27]</td>
				<td><pre>us-core-1: 'SHOULD have a translation to the NDC value set' Rule 'SHOULD have a translation to the NDC value set' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[102].resource.ofType(Immunization).vaccineCode, Line 8026, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-vaccines-cvx (http://hl7.org/fhir/us/core/ValueSet/us-core-vaccines-cvx), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://hl7.org/fhir/sid/cvx#197, http://hl7.org/fhir/sid/ndc#49281012165)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[102].resource.ofType(Immunization).patient, Line 8020, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[102].resource.ofType(Immunization).patient, Line 8020, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[103].resource.ofType(Device), Line 8090, Col 22]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[103].resource.ofType(Device).meta.tag[0], Line 8067, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[103].resource.ofType(Device).type.coding[0], Line 8092, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#109228008 - Code can not be found in CodeSystem for 'http://snomed.info/sct#109228008'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[103].resource.ofType(Device).patient, Line 8085, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[103].resource.ofType(Device).type, Line 8091, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/device-kind (http://hl7.org/fhir/ValueSet/device-kind), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://snomed.info/sct#109228008)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[103].resource.ofType(Device).patient, Line 8085, Col 41]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[103].resource.ofType(Device).patient, Line 8085, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[104].resource.ofType(Condition), Line 8186, Col 36]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[104].resource.ofType(Condition).meta.tag[0], Line 8133, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[104].resource.ofType(Condition).code.coding[0], Line 8169, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#442311008 - Code can not be found in CodeSystem for 'http://snomed.info/sct#442311008'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[104].resource.ofType(Condition).subject, Line 8183, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[104].resource.ofType(Condition).encounter, Line 8179, Col 14]</td>
				<td><pre>Details for Encounter/202201uscore-encounter-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Encounter</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[104].resource.ofType(Condition).code, Line 8168, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-condition-code (http://hl7.org/fhir/us/core/ValueSet/us-core-condition-code), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://snomed.info/sct#442311008)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[104].resource.ofType(Condition).subject, Line 8183, Col 41]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[104].resource.ofType(Condition).subject, Line 8183, Col 41]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[105].resource.ofType(Condition), Line 8271, Col 36]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[105].resource.ofType(Condition).meta.tag[0], Line 8222, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[105].resource.ofType(Condition).code.coding[0], Line 8257, Col 20]</td>
				<td><pre>Unable to validate code http://snomed.info/sct#247824007 - Code can not be found in CodeSystem for 'http://snomed.info/sct#247824007'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[105].resource.ofType(Condition).subject, Line 8268, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[105].resource.ofType(Condition).code, Line 8256, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-condition-code (http://hl7.org/fhir/us/core/ValueSet/us-core-condition-code), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://snomed.info/sct#247824007)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[105].resource.ofType(Condition).subject, Line 8268, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[105].resource.ofType(Condition).subject, Line 8268, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[106].resource.ofType(DiagnosticReport), Line 8342, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[106].resource.ofType(DiagnosticReport).meta.tag[0], Line 8307, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[106].resource.ofType(DiagnosticReport).category[0].coding[0], Line 8315, Col 23]</td>
				<td><pre>Unable to validate code http://loinc.org#LP29684-5 - Code can not be found in CodeSystem for 'http://loinc.org#LP29684-5'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[106].resource.ofType(DiagnosticReport).code, Line 8325, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/ValueSet/report-codes (http://hl7.org/fhir/ValueSet/report-codes), and a coding is recommended to come from this value set) (codes = http://loinc.org#30746-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[106].resource.ofType(DiagnosticReport).code.coding[0], Line 8326, Col 20]</td>
				<td><pre>Unable to validate code http://loinc.org#30746-2 - Code can not be found in CodeSystem for 'http://loinc.org#30746-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[106].resource.ofType(DiagnosticReport).subject, Line 8344, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[106].resource.ofType(DiagnosticReport).category[0], Line 8313, Col 17]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-category (http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-category), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#LP29684-5)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[106].resource.ofType(DiagnosticReport).code, Line 8325, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-report-and-note-codes (http://hl7.org/fhir/us/core/ValueSet/us-core-diagnosticreport-report-and-note-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://loinc.org#30746-2)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[106].resource.ofType(DiagnosticReport).subject, Line 8344, Col 14]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[106].resource.ofType(DiagnosticReport).subject, Line 8344, Col 14]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[107].resource.ofType(MedicationRequest), Line 8420, Col 25]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[107].resource.ofType(MedicationRequest).meta.tag[0], Line 8371, Col 20]</td>
				<td><pre>A code with no system has no defined meaning. A system should be provided</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[107].resource.ofType(MedicationRequest).subject, Line 8421, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/StructureDefinition/Patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[107].resource.ofType(MedicationRequest).dispenseRequest.quantity, Line 8386, Col 32]</td>
				<td><pre>Unable to validate code http://unitsofmeasure.org#mL - Code can not be found in CodeSystem for 'http://unitsofmeasure.org#mL'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Bundle.entry[107].resource.ofType(MedicationRequest).medication.ofType(CodeableConcept), Line 8406, Col 27]</td>
				<td><pre>None of the codings provided are in the value set http://hl7.org/fhir/us/core/ValueSet/us-core-medication-codes (http://hl7.org/fhir/us/core/ValueSet/us-core-medication-codes), and a coding should come from this value set unless it has no suitable code (note that the validator cannot judge what is suitable) (codes = http://www.nlm.nih.gov/research/umls/rxnorm#582620)</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Bundle.entry[107].resource.ofType(MedicationRequest).subject, Line 8421, Col 38]</td>
				<td><pre>Unable to find a match for profile Patient/202201uscore-patient-example-1 among choices: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Bundle.entry[107].resource.ofType(MedicationRequest).subject, Line 8421, Col 38]</td>
				<td><pre>Details for Patient/202201uscore-patient-example-1 matching against Profile http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient</pre></td>
			</tr>
		</table>
	</div>

### Write to file

In [40]:
write_file(b_json)

writing to ... r4/Bundle-2022-01-US-Core-upload.json


### print out FHIR_id Mapping table

In [41]:
my_table = '|US Core IG FHIR ID|US Core Connectathon 29 FHIR ID|\n|---|---|'
for k,v in ref_map.items():
    my_table = (f'{my_table}\n| {k} | {v} |')
display(Markdown(my_table))
print(my_table)

|US Core IG FHIR ID|US Core Connectathon 29 FHIR ID|
|---|---|
| Observation/respiratory-rate | Observation/202201uscore-observation-example-1 |
| RelatedPerson/shaw-niece | RelatedPerson/202201uscore-relatedperson-example-1 |
| Encounter/delivery | Encounter/202201uscore-encounter-example-1 |
| Observation/urine-bilirubin | Observation/202201uscore-observation-example-2 |
| Observation/urine-leukocyte-esterase | Observation/202201uscore-observation-example-3 |
| MedicationRequest/uscore-mo2 | MedicationRequest/202201uscore-medicationrequest-example-1 |
| ServiceRequest/foodpantry-referral | ServiceRequest/202201uscore-servicerequest-example-1 |
| Observation/neutrophils | Observation/202201uscore-observation-example-4 |
| Observation/urine-cells | Observation/202201uscore-observation-example-5 |
| Observation/ekg-lead | Observation/202201uscore-observation-example-6 |
| Observation/blood-pressure | Observation/202201uscore-observation-example-7 |
| Observation/urine-rbcs | Observation/202201uscore-observation-example-8 |
| Organization/acme-lab | Organization/202201uscore-organization-example-1 |
| Observation/urine-glucose | Observation/202201uscore-observation-example-9 |
| Observation/pulse-intensity-palpation | Observation/202201uscore-observation-example-10 |
| Goal/goal-sdoh-2 | Goal/202201uscore-goal-example-1 |
| Observation/qrs-dur-ekg-lead | Observation/202201uscore-observation-example-11 |
| Observation/urine-color | Observation/202201uscore-observation-example-12 |
| Observation/temperature | Observation/202201uscore-observation-example-13 |
| Procedure/rehab | Procedure/202201uscore-procedure-example-1 |
| Observation/satO2-fiO2 | Observation/202201uscore-observation-example-14 |
| Observation/sexual-orientation-example | Observation/202201uscore-observation-example-15 |
| AllergyIntolerance/example | AllergyIntolerance/202201uscore-allergyintolerance-example-1 |
| Observation/serum-potassium | Observation/202201uscore-observation-example-16 |
| Observation/some-day-smoker | Observation/202201uscore-observation-example-17 |
| CareTeam/example | CareTeam/202201uscore-careteam-example-1 |
| Media/ekg-strip | Media/202201uscore-media-example-1 |
| Observation/serum-sodium | Observation/202201uscore-observation-example-18 |
| Observation/serum-creatinine | Observation/202201uscore-observation-example-19 |
| DiagnosticReport/bone-density-report | DiagnosticReport/202201uscore-diagnosticreport-example-1 |
| Observation/oxygen-saturation | Observation/202201uscore-observation-example-20 |
| Location/hospital | Location/202201uscore-location-example-1 |
| Observation/pediatric-wt-example | Observation/202201uscore-observation-example-21 |
| DiagnosticReport/metabolic-panel | DiagnosticReport/202201uscore-diagnosticreport-example-2 |
| Observation/jugular-vein-distension | Observation/202201uscore-observation-example-22 |
| Observation/dxa-hip-l-armass-bmd | Observation/202201uscore-observation-example-23 |
| Observation/hunger-question-example | Observation/202201uscore-observation-example-24 |
| Observation/prapare-multiselect-example | Observation/202201uscore-observation-example-25 |
| Observation/ofc-percentile | Observation/202201uscore-observation-example-26 |
| Observation/urine-hemoglobin | Observation/202201uscore-observation-example-27 |
| Organization/saint-luke-w-endpoint | Organization/202201uscore-organization-example-2 |
| DocumentReference/episode-summary | DocumentReference/202201uscore-documentreference-example-1 |
| Observation/length | Observation/202201uscore-observation-example-28 |
| Observation/urine-protein | Observation/202201uscore-observation-example-29 |
| Observation/urobilinogen | Observation/202201uscore-observation-example-30 |
| Encounter/1036 | Encounter/202201uscore-encounter-example-2 |
| DiagnosticReport/cardiology-report | DiagnosticReport/202201uscore-diagnosticreport-example-3 |
| Practitioner/practitioner-1 | Practitioner/202201uscore-practitioner-example-1 |
| CarePlan/colonoscopy | CarePlan/202201uscore-careplan-example-1 |
| Observation/serum-calcium | Observation/202201uscore-observation-example-31 |
| DiagnosticReport/urinalysis | DiagnosticReport/202201uscore-diagnosticreport-example-4 |
| Medication/uscore-med2 | Medication/202201uscore-medication-example-1 |
| Device/udi-1 | Device/202201uscore-device-example-1 |
| Observation/ekg-impression | Observation/202201uscore-observation-example-32 |
| Observation/urine-ph | Observation/202201uscore-observation-example-33 |
| Condition/condition-SDOH-example | Condition/202201uscore-condition-example-1 |
| Medication/uscore-med1 | Medication/202201uscore-medication-example-2 |
| Device/udi-2 | Device/202201uscore-device-example-2 |
| Goal/goal-1 | Goal/202201uscore-goal-example-2 |
| Patient/example | Patient/202201uscore-patient-example-1 |
| Observation/height | Observation/202201uscore-observation-example-34 |
| Observation/urine-sediment | Observation/202201uscore-observation-example-35 |
| Patient/infant-example | Patient/202201uscore-patient-example-2 |
| Observation/blood-glucose | Observation/202201uscore-observation-example-36 |
| Practitioner/practitioner-2 | Practitioner/202201uscore-practitioner-example-2 |
| DiagnosticReport/cbc | DiagnosticReport/202201uscore-diagnosticreport-example-5 |
| MedicationRequest/self-tylenol | MedicationRequest/202201uscore-medicationrequest-example-2 |
| Observation/heart-rate-rhythm | Observation/202201uscore-observation-example-37 |
| Encounter/example-1 | Encounter/202201uscore-encounter-example-3 |
| Observation/urine-nitrite | Observation/202201uscore-observation-example-38 |
| Patient/child-example | Patient/202201uscore-patient-example-3 |
| Location/hl7east | Location/202201uscore-location-example-2 |
| Observation/socialhistory-assessment-example | Observation/202201uscore-observation-example-39 |
| Procedure/defib-implant | Procedure/202201uscore-procedure-example-2 |
| Immunization/imm-1 | Immunization/202201uscore-immunization-example-1 |
| Device/udi-3 | Device/202201uscore-device-example-3 |
| Observation/urine-epi-cells | Observation/202201uscore-observation-example-40 |
| Observation/head-circumference | Observation/202201uscore-observation-example-41 |
| Observation/heart-rate | Observation/202201uscore-observation-example-42 |
| Condition/example | Condition/202201uscore-condition-example-2 |
| Observation/urine-clarity | Observation/202201uscore-observation-example-43 |
| Observation/dxa-femur-l-t-score-bmd | Observation/202201uscore-observation-example-44 |
| Observation/p-r-interval-ekg-lead | Observation/202201uscore-observation-example-45 |
| Observation/dxa-femur-l-armass-bmd | Observation/202201uscore-observation-example-46 |
| Observation/q-t-interval-ekg-lead | Observation/202201uscore-observation-example-47 |
| Condition/hc1 | Condition/202201uscore-condition-example-3 |
| Observation/bmi | Observation/202201uscore-observation-example-48 |
| DiagnosticReport/chest-xray-report | DiagnosticReport/202201uscore-diagnosticreport-example-6 |
| Observation/serum-chloride | Observation/202201uscore-observation-example-49 |
| Observation/hemoglobin | Observation/202201uscore-observation-example-50 |
| Observation/capillary-refill-time-nail-bed | Observation/202201uscore-observation-example-51 |
| Observation/bp-data-absent | Observation/202201uscore-observation-example-52 |
| Organization/example-organization-2 | Organization/202201uscore-organization-example-3 |
| Observation/usg | Observation/202201uscore-observation-example-53 |
| Observation/dxa-hip-l-t-score-bmd | Observation/202201uscore-observation-example-54 |
| Observation/dxa-femur-l-z-score-bmd | Observation/202201uscore-observation-example-55 |
| Observation/urine-ketone | Observation/202201uscore-observation-example-56 |
| Observation/serum-total-bilirubin | Observation/202201uscore-observation-example-57 |
| Observation/urine-wbcs | Observation/202201uscore-observation-example-58 |
| Observation/weight | Observation/202201uscore-observation-example-59 |
| Observation/pediatric-bmi-example | Observation/202201uscore-observation-example-60 |
| Observation/erythrocytes | Observation/202201uscore-observation-example-61 |
| Observation/bun | Observation/202201uscore-observation-example-62 |
| Observation/serum-co2 | Observation/202201uscore-observation-example-63 |
| Observation/urine-bacteria | Observation/202201uscore-observation-example-64 |
| Observation/dxa-hip-l-z-score-bmd | Observation/202201uscore-observation-example-65 |
| Observation/mchc | Observation/202201uscore-observation-example-66 |
| MedicationRequest/uscore-mo1 | MedicationRequest/202201uscore-medicationrequest-example-3 |

|US Core IG FHIR ID|US Core Connectathon 29 FHIR ID|
|---|---|
| Observation/respiratory-rate | Observation/202201uscore-observation-example-1 |
| RelatedPerson/shaw-niece | RelatedPerson/202201uscore-relatedperson-example-1 |
| Encounter/delivery | Encounter/202201uscore-encounter-example-1 |
| Observation/urine-bilirubin | Observation/202201uscore-observation-example-2 |
| Observation/urine-leukocyte-esterase | Observation/202201uscore-observation-example-3 |
| MedicationRequest/uscore-mo2 | MedicationRequest/202201uscore-medicationrequest-example-1 |
| ServiceRequest/foodpantry-referral | ServiceRequest/202201uscore-servicerequest-example-1 |
| Observation/neutrophils | Observation/202201uscore-observation-example-4 |
| Observation/urine-cells | Observation/202201uscore-observation-example-5 |
| Observation/ekg-lead | Observation/202201uscore-observation-example-6 |
| Observation/blood-pressure | Observation/202201uscore-observation-example-7 |
| Observation/urine-rbcs | Observation